# MA6202: Laboratorio de Ciencia de Datos

**Profesor: Nicolás Caro**

**19/06/2020 - C12 S7**

# Redes Neuronales 

## Introducción
La redes neuronales aparecen bajo el nombre de **perceptron**, propuesta por [Rosenblatt en 1958](http://www.ling.upenn.edu/courses/cogs501/Rosenblatt1958.pdf). Este es un modelo de clasificación consistente en pesos $w$ y una función de output del tipo $f(w^t\mathbf{x})$ para el input $\mathbf{x}$. Si bien este modelo es en esencia muy similar a la regresión logística, el percetron implementa $f(\cdot)$ como una función de salto, que entrega el valor 1 para argumentos mayores que 0 y retorna 0 en otro caso. El problema de este modelo consiste en su poca expresibilidad, pues solo permite hacer separaciones lineales. 

Sobre el modelo anterior, se construye el perceptron multicapa, el cual consiste en múltiples modelos de perceptron intecomunicadas, cada uno de estos modelos se denota como *unidad* (*unit*) y se organizan en capas secuenciales conocidas como *capa input*, corresponde a un conjunto de perceptrones que operan de manera paralela sobre un vector input. El resultado de sus clasificaciones corresponde a un vector, donde cada componente se asocia un perceptron de la capa input. Las siguientes capas se denotan como *capas ocultas* y se contruyen de manera análoga a la capa input (o inicial), la primera capa oculta consiste en un conjunto de percetrones que operan sobre el output de la capa input. Sobre esta primera capa oculta pueden haber multiples capas ocultas que operan sobre el output de la capa oculta anterior. El proceso de generar una salida sobre la entrada de una capa anterior se denota *propagación hacia adelante* (*feedforward*) la propagación termina en una última capa denominada *capa output* que entrega el resultado final de la clasificación. 

Bajo el punto de vista del aprendizaje automático, las capas ocultas de un perceptron multicapa (red neuronal) generan abstracciones o características a partir de los datos sobre los cuales operan. 

Para trabajar con redes neuronales haremos uso de la librería **Pytorch**. Esta librería consiste en un conjunto de herramientas diseñadas para generar modelos basados en redes neuronales utilizando las capacidades de computo distribuido que ofrecen las GPU (unidades de procesamiento gráfico / tarjetas de video). Esto permite operaciones de vectorización aceleradas y distribuidas. Esta librería se importa como `torch`.

**Ejemplo**

Se implementa una red neuronal simple utilizando Pytorch. Para esto, se importa el módulo y se indica una semilla aleatoria.

In [ ]:
import torch

# Se asigna un valor de reproductibilidad
torch.manual_seed(6202)

Se carga el dataset sobre el cual trabajaremos, en este caso será un conjunto de datos de vinos. Este conjunto consta de 13 atributos continuos, cada vino posee un identificador dentro de 3 posibilidades, la idea es poder clasificar cada vino para predecir su identificador. 

In [ ]:
import pandas as pd

names = [
    'class','Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium',
    'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
    'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline'
]

wine_data = pd.read_csv(
    'http://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data',
    names=names)

wine_data.head()

Se procede a hacer una separación en entrenamiento y test, se hará es una codificación dummy para la variable de respuesta y se estandarizan las variables numéricas

In [ ]:
from sklearn.preprocessing import OrdinalEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

response = ['class']
num_cols = wine_data.loc[:, 'Alcohol':].columns.values

data_transform = ColumnTransformer(
    transformers=[('normaliza', StandardScaler(),num_cols), 
                  ('codifica', OrdinalEncoder(), response)])

Para evitar fuga de información, se hace una separación en train y test para luego transformar los datos

In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(wine_data, test_size=.2)

Se aplica el preprocesamiento sobre el conjunto train

In [ ]:
data = data_transform.fit_transform(data_train)

# Se obtienen las variables numericas y de respuesta

X_train = data[:,:-1].copy()
y_train = data[:,-1:].copy()

El perceptron multicapa a modelar consiste en 10 capas ocultas y su función de activación tiene la forma $f(x) = max(0,x)$, esta es una función conocida y se denota como ReLU (Rectified Linear Unit). Se definen los parámetros, para la función de activación se utiliza el módulo de redes neuronales de Pytorch, se puede acceder a este módulo por medio de `torch.nn`.

In [ ]:
input_dim, output_dim = (X_train.shape[1], 3)

# Equivalente al output de la capa input
input_dim_capas_ocultas = 5

f_activation = torch.nn.ReLU()

Para definir la primera capa, tenemos que tener en cuenta que buscamos una transformación de la forma $x \mapsto w^t x$ que luego es clasificada por medio de $f(w^t x)$. Esto quiere decir que los parámetros a entrenar serán los vectores de peso $w$ asociados a cada unidad (perceptron) de cada capa (input, ocultas y output). Pytorch ofrece una abstracción para capas formadas por perceptrones de la forma $f(w^t x)$. Esta abstracción es la clase `Linear` (observe que $x \mapsto w^t x$ es una transformación lineal), se procede a definir la capa input.




In [ ]:
from torch.nn import Linear

capa_input = Linear(in_features  = input_dim,
                    out_features = input_dim_capas_ocultas)

Luego se definen las capas ocultas, estas también corresponden a transformaciones lineales sobre sus capas predecesoras. En este caso, cada capa oculta tendrá dimensión 5 como input y output. Se define una capa oculta

In [ ]:
Linear(input_dim_capas_ocultas, input_dim_capas_ocultas)

La arquitectura de esta red es de naturaleza secuencial y corresponde al siguiente algoritmo:

1. Input: x de dimensión 13
2. Opera x en cada unidad de la capa lineal input, es decir, calcula $w_i^t x$ para $i = 1, \ldots, 5$ (dimensión de salida de la capa input).
3. Opera RelU($w_i^t x$) para $i = 1, \ldots, 5$, se obtiene un vector de dimensión 5. 

4. Para cada una de las 2 capas ocultas y de manera secuencial:
    1. Genera una transformación lineal sobre los outputs de la capa anterior.
    2. Aplica ReLU sobre las transformaciones lineales.
    3. En la última capa oculta el vector resultante se pasa a la capa output.

5. Se recibe el resultado de la última capa oculta, se transforma de manera lineal sobre 3 perceptrones para finalmente ser transformada por medio de una función softmax (dimensión del output es 3). 

Esto se reduce a: 
```
X  -> capa input -> ReLU() -> Co_1 -> ReLU() -> Co_2 -> ReLU() -> capa output -> predicción
```

Para programar el esquema anterior se hace uso de un diccionario ordenado `OrderedDict` de la librería `collections`. Este tipo de objetos opera de manera similar a las Pipelines de Scikit-learn, pues reciben un conjunto de tuplas del tipo `(identificador,objeto)`. Se procede a generar la arquitectura de la red:

In [ ]:
from collections import OrderedDict

n_capas_ocultas = 2

# capa input
input_ = ('capa input', capa_input)
relu = ('relu', f_activation)

steps = [input_, relu]

# capas ocultas
for i in range(n_capas_ocultas):
    
    capa = ('capa oculta_' + str(i),
         Linear(input_dim_capas_ocultas, input_dim_capas_ocultas))
    
    steps.extend([capa,relu])

# capa output
output = ('capa ouput', Linear(input_dim_capas_ocultas, output_dim))

steps.extend([(output),('Softmax',torch.nn.Softmax(dim=1))])

# Se utiliza la estructura de diccionario ordenado
steps = OrderedDict(steps)

Luego, cuando ya se posee la arquitectura, se inicializa un objeto `Sequential` del módulo `nn`, este objeto permite modelar una red neuronal multicapa recibiendo como input los componentes de la arquitectura de manera ordenada.

In [ ]:
from torch.nn import Sequential

#MLP -> multi layer perceptron
mlp = Sequential(steps)

El resultado entregado por la capa output corresponde a un vector de tres dimensiones, se asigna la clase predicha a aquella componente con el mayor valor. Luego de definir la red, es necesario definir un criterio de optimalidad (función de perdida), en este caso se utiliza la entropía cruzada. Este criterio permite comparar dos distribuciones de probabilidad $q$ (aproximación) y $p$ (real) en términos de la diferencia de información esperada (en bits por ejemplo) al utilizar la distribución $q$ para describir un eventos codificados, optimizados para $p$. Dado que se trabaja en un problema de clasificación (supervisado) se conoce la distribución real $p$ para una etiqueta  (ej: etiqueta (1,0,0), distribución (100%, 0, 0) ), por otra parte, la distribución aproximada viene dada por nuestro modelo. La entropía cruzada entre $p$ y $q$ se expresa según:
$$
H(p, q)=-\sum_{x \in \mathcal{X}} p(x) \log q(x)
$$

Se implementa mediante:

In [ ]:
criterio = torch.nn.CrossEntropyLoss()

Se debe seleccionar el optimizador a utilizar, en este caso será **descenso de gradiente estocástico** (SGD, se estudiará con más detenimiento posteriormente). Este optimizador tiene un parámetro asociado a la proporción de aprendizaje (*learning rate*) y momentum. Se inicializa entregando dichos parámetros y los coeficientes sobre los que opera, en este caso, los parámetros de la red `mpl` a los cuales se acceede por medio del método `.parameters()`

In [ ]:
optimizador = torch.optim.SGD(mlp.parameters(), lr=0.05, momentum=0.1)

Finalmente, se puede entrenar la red definida, para ello se define una cantidad de *épocas* (*epochs*), esto se refiere a la cantidad de veces que se entrena utilizando el conjunto de entrenamiento. Este proceso tiene el siguiente orden:

1. Genera un conjunto de inputs en un formato compatible.
2. En cada época:
    1. Inicializa los gradientes asociados al optimizador, esto evita que se acumulen gradientes entre épocas.
    2. Se opera sobre los inputs para obtener las predicciones.
    3. Se calcula el error de predicción.
    4. Se propagan los valores de la función de perdida para ajustar los pesos de la red y mejorar predicciones. Este proceso se denomina como *propagación hacia atrás* (*backpropagation*).
    5. Se pasa a la siguiente época.
    
Se implementa el esquema anterior:

In [ ]:
#Paso A
datos_input = torch.autograd.Variable(torch.Tensor(X_train))
labels = torch.autograd.Variable(torch.Tensor(y_train.reshape([-1,])).long())

In [ ]:
epocas = 1000
for ep in range(epocas):
    #Paso B
    optimizador.zero_grad()
    for i in range(20):
        #Paso C
        out = mlp(datos_input)
        out.requires_grad_(True)

        #Paso D 
        loss = criterio(out, labels)
        
        #Paso E
        loss.backward() 
        optimizador.step()
    
    if ((ep+1) % 100) == 0 : print('Epoca: ', ep+1, 'Loss: ', loss.data)

Se estudia el rendimiento en train y test

In [ ]:
from sklearn.metrics import classification_report
import numpy as np

dt_test = data_transform.transform(data_test)

X_test = dt_test[:, :-1]
y_test = dt_test[:, -1:]

f = lambda x: torch.argmax(mlp(x), dim=1)

#Train error
datos_input = torch.Tensor(X_train).float()
preds_train = f(datos_input)

print('Reporte train : \n',
      classification_report(y_train.reshape([-1,]), preds_train))

In [ ]:
# Test Error
datos_input = torch.Tensor(X_test).float()
preds_test = f(datos_input)

print('Reporte test : \n', 
      classification_report(y_test.reshape([-1,]), preds_test))

se puede decir que el clasificador entrenado fue un éxito.

Las redes neuronales pueden ser descritas como un modelo matemático de procesamiento de información. En general, una red neuronal puede considerarse como un sistema con las siguientes características:

1. El procesamiento de la información ocurre en unidades llamadas neuronas.
2. Las neuronas están conectadas e intercambian información (o señales) por medio de sus conexiones.
3. Las conexiones entre neuronas pueden ser fuertes o débiles, dependiendo de como se procesa la información.
4. Cada neurona tiene un estado interno determinado por todas las conexiones que posee.
5. Cada neurina tiene una función de activación que opera sobre su estado, esta función determina la información que se comparte a otras neuronas.

En términos operativos, una red neuronal posee una **arquitectura** que describe el conjunto de conexiones entre neurona y un proceso de **aprendizaje** asociado, que describe el proceso entrenamiento.

Las **neuronas** por tanto, pueden ser definidas por medio de la siguiente relación:
$$
y=f\left(\sum_{i} x_{i} w_{i}+b\right)
$$

Acá se hace el calculó $w^t x + b = \sum_i x_i w_i + b$ sobre los inputs $x_i$ y los pesos $w_i$. Estos últimos, son valores numéricos que representan las conexiones entre neuronas, el peso $b$ se denomina *bias*. Luego se calcula el resultado de aplicar la función de activación $f(\cdot)$. Existen distintos tipos de funciones de activación dentro de estas se pueden nombrar:

* $f(x)=x$ la función identidad.
* $f(x)=\left\{\begin{array}{l}1 \text { if } x \geq 0 \\ 0 \text { if } x<0\end{array}\right.$ la función de activación de umbral.
* $f(x)=\frac{1}{1+\exp (-x)}$ la función sigmoide logistica, es una de las más utilizadas.
* $f(x) =\frac{1-\exp (-x)}{1+\exp (-x)}$ la función sigmoide bipolar, esta corresponde a una sigmoide escalada a $(-1,1)$.
* $f(x) = \frac{1-\exp (-2 x)}{1+\exp (-2 x)}$ la tangente hiperbólica. 

* $f(x)=\left\{\begin{array}{l}x\text { if } x \geq 0 \\ 0 \text { if } x<0\end{array}\right.$ La función de activación ReLU.

**Ejercicio**

1. Existen variantes de la función de activación ReLU, investigue al menos 3 y compare sus diferencias.

Por su parte, las **capas de una red** conforman su arquitectura al agrupar conjuntos de neuronas. Acá se puede distinguir la *capa input* que representa las condiciones iniciales del dataset. La *capa output* puede tener una neurona (ej: problemas de regresión) o más (como en el ejemplo anterior). Si hay capas entre el input y el output, estas se denominan *capas ocultas*. En general las capas pueden estar completamente conectadas (como en el ejemplo implementado) o pueden faltar conexiones entre neuronas, esto se puede representar por los valores de los pesos, por ejemplo, el si denotamos $w_{ij}^{(k,k+1)}$ como los pesos entre el output de la neurona $i$ en la capa $k$ con el input de la neurona $j$ de la capa $k+1$, un peso $w_{ij}^{(k,k+1)} = 0$ implica que la neurona $j$ esima de la capa $k+1$ no posee interacción con la neurona $i$ esima de la capa $k$. 

**Ejercicios**

1. El esquema presentado en el ejemplo anterior implica que las capas de una red deben conectarse de manera secuencial. En general esto no tiene porque ser cierto, en efecto, las conexiones entre capas pueden ser modeladas por medio de grafos *acíclicos dirigidos*. ¿Por qué es necesario utilizar grafos de este tipo? 

2. El teorema de aproximación universal para redes neuronales dice que toda función continua en un compacto de $\mathbb{R}^n$ puede ser aproximada por una red neuronal con al menos una capa oculta. Para tener una intuición sobre esto, observe que es posible aproximar funciones continuas por medio de sumas ponderadas de *funciones boxcar* o *cajón*, estás, para una altura $A$ y base $(b-a)$ , $b >a$ se puede escribir como $\operatorname{boxcar}(x)=A(H(x-a)-H(x-b))$, donde:
$$
H(x)=\left\{\begin{array}{ll}0, & x<0 \\ 1, & x\geq 0\end{array}\right.
$$ 

Implemente una función boxcar utilizando una red neuronal con inputs en $x \in \mathbb{R}$, una capa oculta de dos neuronas, cada una con función de activación $f(x) = 1 /(1+\exp (-x))$, la red genera un output $y \in \mathbb{R}$.

El proceso de aprendizaje en redes pasa por definir un **esquema de entrenamiento**. Esto hace referencia a encontrar un conjunto de conexiones óptimas entre las las neuronas de la arquitectura para generar predicciones. Estas conexiones optimas se representan por los pesos de la red en cada capa y se entrenan en función de algún esquema de optimización (Descenso de gradiente por ejemplo).

Observemos por ejemplo una regresión lineal, en este caso la red consta de una capa oculta, un output unidimensional y la función de activiación identidad. Es posible entrenar una regresión lineal utilizando descenso de gradiente, para ello se define el función a minimizar $J$ como:
$$
J = \frac{1}{n} \sum_{i=0}^{n}\left(y_{i}-t_{i}\right)^{2}=\frac{1}{n} \sum_{i=0}^{n}\left(w^t x_{i} - {t_i}\right)^{2}
$$

En este caso $y_i$ representa el output del modelo lineal. El funcional a minimizar es el error cuadrátrico medio. El esquema de descenso de gradiente para este modelo pasa a ser:
$$
w \rightarrow w-\lambda \nabla(J(w))
$$

El cual se detiene al llegar a cierto umbral para $J$ o luego de cierto número de iteraciones. 

**Ejercicios**

1. Encuentre una expresión exacta para $\nabla(J(w))$ e implemente un esquema de descenso de gradiente para la regresión lineal. ¿Qué nombre se asocia al parámetro $\lambda$ en el esquema $w \rightarrow w-\lambda \nabla(J(w))$?¿Cuál es su función?

2. El esquema de regresión logística corresponde a una red neuronal idéntica a la utilizada para regresión lineal, con la salvedad de que posee una función de activación sigmoide logística. Calcule un descenso de gradiente para este tipo de red. 

Se debe observar que al implementar el esquema anterior, se produce una acumulación de error el todo en conjunto de entrenamiento. En conjunto de datos grandes esto no es práctico. Una solución es implementar *Descenso de Gradiente Estocástico*, este algoritmo consiste en aproximar los pesos del esquema anterior sobre un subconjunto de observaciones del conjunto de entrenamiento, este subconjunto se denota como **mini -batch**.

Las redes de una capa oculta (regresión logística y lineal) poseen reglas sencillas de actualización según la función de activación, con esto se pueden actualizar los pesos de cada conexión utilizando el error de predicción como referencia. En redes con múltiples capas sólo se puede aplicar esta técnica a las neuronas de la última capa oculta, esto pues, no se tienen valores reales (*target*) con los cuales comparar los resultados de las capas ocultas intermedias. En este caso, lo que se hace es calcular el error de la ultima capa oculta para luego hacer una **estimación** del error cometido en la capa oculta anterior, este proceso se hace de manera recurrente hasta llegar a la capa inicial. Tal proceso de denota como *propagación hacia atrás* (**backpropagation**). Este algoritmo es la base del aprendizaje con redes neuronales. Para comprenderlo es necesario definir los siguientes conceptos:


1. Se define $w_{ij}$ como el peso entre la neurona $i$ esima de la capa $l$ con la neurona $j$ esima de la capa $l+1$. Es decir, el subindice izquierdo hace referencia a una capa anterior a la que neurona que representa el subindice derecho. Al convención $l$ hace referncia a cualquier capa oculta de la red.

2. Se utiliza $y$ para denotar inputs y outputs entre capas.  Es decir, $y_i$ es el input de la capa $l+1$ pero también el output de la capa $l$. 

3. El funcional de pérdida se denota como $J$, el valor de activación $w^t x$ se denota como $a$.

4. $a_j$ es una función de activación con pesos $w_j$, $y_j$ es una función de $a_j$ y $J$ es función de $y_j$. Según la regla de la cadena se tiene así:
$$
\frac{\partial J}{\partial w_{i, j}}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} \frac{\partial a_{j}}{\partial w_{i, j}}
$$

5. Como se sabe que $\frac{\partial a_{j}}{\partial w_{i, j}}=y_{i}$, se tiene:

$$
\frac{\partial J}{\partial w_{i, j}}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} y_{i}
$$

6. Para las capas anteriores se cumple la misma formula (por notación):$$
\frac{\partial J}{\partial w_{i, j}}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} \frac{\partial a_{j}}{\partial w_{i, j}}
$$
Aunque se tengan muchas capas, siempre es posible concentrarse en pares de capas consecutivas, por que se logra siempre tener una capa input y una capa output. Como se sabe que $\frac{\partial a_{j}}{\partial w_{i, j}}=y_{i}$, es decir, la derivada parcial del valor de  activación de la neurona $j$-ésima en la capa $l+1$ con respecto al peso de la conexión del output $y_j$ entre las capas $l$ y $l+1$ con la neurona $i$-ésima, es justamente el valor del output sin ser ponderado por el peso de la conexión. Más aún, $\frac{\partial y_{j}}{\partial a_{j}}$ es la derivada de la función de activación que se puede calcular. Entonces, en la expresión obtenida por la regla de la cadena, sólo hace falta calcular $\frac{\partial J}{\partial y_{j}}$, la derivada del error con respecto a la función de activación en la segunda capa. Con esto, se pueden calcular todas las derivadas partiendo desde la última capa hacia atrás pues:
    * Se puede calcular esta derivada para la última capa
    * Se tiene la formula que permite calcula la derivdada de una capa, teniendo calculada la derivada de la capa siguiente. 

**Ejemplo**

En la siguiente ecuación, $y_i$ es el output de la primera capa, $y_j$ el ouput de la segunda capa. Aplicando la regla de la cadena se tiene:
$$
\frac{\partial J}{\partial y_{i}}=\sum_{j} \frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial y_{i}}=\sum_{j} \frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} \frac{\partial a_{j}}{\partial y_{i}}
$$

Se puede calcular $\frac{\partial y_{j}}{\partial a_{j}}$ y $\frac{\partial a_{j}}{\partial y_{i}}=w_{i, j}$ , si $\frac{\partial J}{\partial y_{j}}$ es conocido (gradiente en la capa segunda), se puede calcular $\frac{\partial J}{\partial y_{i}}$ (gradiente en la capa primera). En resumen, si se tienen 3 capas conectadas de manera secuencia según los inputs (outputs) $y_i \rightarrow y_j \rightarrow y_k$ entonces se pueden aplicar las ecuaciones:

$$
\frac{\partial J}{\partial w_{i, j}}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} \frac{\partial a_{j}}{\partial w_{i, j}}
$$

y 

$$
\frac{\partial J}{\partial y_{j}}=\sum_{k} \frac{\partial J}{\partial y_{k}} \frac{\partial y_{k}}{\partial y_{j}}
$$

Para calcular las derivadas del costo en cada capa. 

Al definir $\delta_{j}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}}$, se tiene que $\delta_j$ representa la variación de costo con respecto a cada valor de activación. Con esto se puede reescribir:

$$
\frac{\partial J}{\partial y_{i}}=\sum_{j} \frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial y_{i}}=\sum_{j} \frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}} \frac{\partial a_{j}}{\partial y_{i}}=\sum_{j} \delta_{j} w_{i, j}
$$

Con lo que $\delta_{i}=\left(\sum_{j} \delta_{j} w_{i, j}\right) \frac{\partial y_{i}}{\partial a_{i}}$. Esto se traduce en poder calcular la variación para la siguiente capa:

$$\delta_{j}=\frac{\partial J}{\partial y_{j}} \frac{\partial y_{j}}{\partial a_{j}}$$

$$\delta_{i}=\left(\sum_{j} \delta_{j} w_{i, j}\right) \frac{\partial y_{i}}{\partial a_{i}}$$

Combinando las ecuaciones:

$$
\frac{\partial J}{\partial w_{i, j}}=\delta_{j} \frac{\partial a_{j}}{\partial w_{i, j}}=\delta_{j} y_{i}
$$

Con lo que la regla de actualización para los pesos de cada capa pasa a ser: 

$$
w_{i, j} \rightarrow w_{i, j}-\eta \delta_{j} y_{i}
$$

En Pytorch es posible implementar reglas de actualización basadas en propagación hacia atrás de manera automática. Para ello, es necesario comprender los tipos de datos asociados a esta librería. 

En primer lugar, se tienen los **tensores**, estos son arreglos n dimensionales, soportan almacenamiento en GPU. Las operaciones y sintaxis de este tipo de objetos es muy similar a los arreglos de NumPy. 

**Ejemplo**

Pytorch se mimetiza con NumPy en cuanto a la definición de sus métodos y compatibilidad cruzada, por ejemplo, para definir un arreglo de 2x2 se puede utilizar la función `randn` (análoga a la de NumPy)

In [ ]:
x = torch.randn(2, 2)
x

Se pueden importar arreglos de numpy de manera bastante sencilla

In [ ]:
x = np.random.randn(2,2)
x = torch.from_numpy(x)
x

La función `torch.cuda.is_available()` permite detectar si existe una GPU disponble para computo. En caso afirmativo, se pueden cargar tensores en la GPU para su posterior computo, para ello, se utiliza la función `torch.device()` y el método `.to()` de los objetos `Tensor`.

**Ejemplo**

Se implementan operaciones en la GPU, para ello se define un tensor vacio `x`

In [ ]:
x = torch.empty((2,2))

se modifica `x` de manera inplace en la CPU

In [ ]:
x.add_(1)

Se define un flujo en la CPU

In [ ]:
# Se detecta la presencia de una GPU
if torch.cuda.is_available():
    # Se obtiene un objeto que apunta a la GPU descubierta
    device = torch.device("cuda")          
    
    # Se crea un tensor y se carga en la GPU
    y = torch.ones_like(x, device=device)  #
    
    # Se envia un vector a la GPU luego de ser definido
    x = x.to(device)                       
    
    # Se obtiene un resultado en la GPU y se transfiere a la CPU
    z = x + y
    print('z en GPU: \n', z)
    print('z en CPU:\n', z.to("cpu", torch.double))

**Ejercicio**

1. ¿Qué notación común siguen las operaciones *in place* para tensores?

El motor central en las redes neuronales de PyTorch es la librería  `autograd`. Esta entrega herramientas de diferenciacion automatica para todas las operaciones hechas sobre objetos tipo `Tensor`. 

Como se estudio anteriormente, los tensores son objetos similares a los arreglos de NumPy en cuanto a sus métodos y manejo. Aparte de permitir el computo en GPU, poseen el atributo  el atributo booleano `.requires_grad`. Si para un tensor, tal atributo tiene el valor `True`, se comienzan a registrar todas operaciones aplicadas aplicadas sobre este. Tal registro se lleva a cabo de manera a automática, generando una estructura jeraquica codificada en un grafo denotado como DCG (grafo dinámico computacional). Este grafo es grafo acíclico y dirigido, sus hojas son en efecto, los tensores siendo seguidos (*input*) y las raíces son los tensores de respuesta luego de la última operación (*output*). Tal grafo permite calcular gradientes siguiendo un esquema similar al de de backpropagation, donde se multiplican gradientes en cada etapa desde la raíz al cierto nodo hoja de interés.

Para generar el registro antes descrito, las operaciones realizadas sobre un tensor input son asociadas a una operación por medio de la clase `Function`. Cada tensor del cual se lleva un registro, tendrá el atributo `.grad_fn` que referencia al objeto de clase `Function` que dio origien a tal tensor. En el caso de tensores creados explícitamente, `.grad_fn`será `None`. 

**Ejemplo**

Se crea un grafo usando tensores que requieren gradiente.

In [ ]:
x = torch.tensor(5.0, requires_grad=True)
y = torch.tensor(1.0)
z = x * y + 1

# Componentes del grafo
for i, name in zip([x, y, z], "XYZ"):

    print(f"{name}\n Datos: {i.data}\n requires_grad: {i.requires_grad}\n\
 grad: {i.grad}\n grad_fn: {i.grad_fn}\n\n ¿Es hoja? : {i.is_leaf}\n")

Para hacer operaciones sin registrar gradientes se utiliza el context manager `torch.no_grad()`.

In [ ]:
x = torch.tensor(25.0, requires_grad=True)
y = x * 2

print('Se requiere gradiente en y:', y.requires_grad)

with torch.no_grad():
    y = x * 2
    print('Se requiere gradiente en y (context manager): ', y.requires_grad)

**Ejercicio**

1. ¿Qué función tiene el método `.detach()` en un tensor?

El método `.backward()` calcula los gradientes al aplicarse sobre un **valor escalar** (tensor unitario) es,to se hace recorriendo el grafo generado por las operaciones siguiendo el esquema de propagación hacia atrás. Los gradientes calculados se guardan en el atributo `.grad` de cada nodo hoja. 

**Ejemplo**

Se estudia el gradiente de una función 

In [ ]:
# Se genera el grafo de manera dinamica en cada operacion
x = torch.tensor(5.0, requires_grad = True)
z = x ** 3

# Se calculan los gradientes
z.backward() 

En el caso anterior, se tiene la función $z = x^3$ cuyo gradiente con respecto a $x$ es $3x^2$ que evaluado en $x=5$ pasa a ser $75$. Se confirma lo anterior

In [ ]:
print(x.grad.data)

Cuando se llama `z.backward()` se pasa como argumento el tensor unitario `torch.tensor(5.0)`, es decir, se calcula `z.backward(torch.tensor(5.0))`, en este caso, dicho argumento es el *gradiente externo* utilizado para terminar las multiplicaciones asociadas a la regla de la cadena. Por ejemplo, si se tienen los tensores :

In [ ]:
x = torch.tensor([2.0, 4.0, 8.0], requires_grad = True)
y = torch.tensor([1.0 , 3.0 , 5.0], requires_grad = True)

z = x*y
z

In [ ]:
z.backward(torch.tensor([1.0,1.0,1.0]))
x.grad

Entonces, para calcular los gradientes de `z` con respecto a `x` o a `y`, se debe entregar un gradiente externo de dimensión 3. 

En términos generales, si se tiene una función vectorial $\vec{y}=f(\vec{x})$, entonces el gradiente de $\vec{y}$ con respecto a $\vec{x}$ serpa la matriz Jacobiana:
$$
J=\left(\begin{array}{ccc}\frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}} \\ \vdots & \ddots & \vdots \\ \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}\end{array}\right)
$$

Para efectuar el algoritmo de propagación hacia atrás, `torch.autograd` calcula productos entre matrices jacobianas y vectores, por lo que dado un vector $v=\left(v_{1} , v_{2} , \ldots , v_{m}\right)^{T}$, se calcula $v^{T} \cdot J$. Si en tal caso, $v$ es el gradiente de una función escalar $l=g(\vec{y})$, es decir, $v=\left( \frac{\partial l}{\partial y_{1}} , \cdots , \frac{\partial l}{\partial y_{m}}\right)^{T}$, entonces por la regla de la cadena, el producto siguiente
$$
J^{T} \cdot v=\left(\begin{array}{ccc}\frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}} \\ \vdots & \ddots & \vdots \\ \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}\end{array}\right)\left(\begin{array}{c}\frac{\partial l}{\partial y_{1}} \\ \vdots \\ \frac{\partial l}{\partial y_{m}}\end{array}\right)=\left(\begin{array}{c}\frac{\partial l}{\partial x_{1}} \\ \vdots \\ \frac{\partial l}{\partial x_{n}}\end{array}\right)
$$

Pasa a ser el gradiente de $l$ con respecto a $\vec{x}$. Tal desarrollo hace que tenga sentido entregar gradientes externos a una cadena de operaciones sin outpus escalares.


**Ejercicio**

1. Estudie la documentación de la clase `Function` perteneciente al módulo `torch.autograd`. [link](https://pytorch.org/docs/stable/autograd.html#function). 

## Aprendizaje Profundo

El aprendizaje profundo o *deep learning* consiste en la aplicación de modelos basados en redes neuronales con múltiples capas ocultas. Este tipo de redes presenta la ventaja pues sus estructuras no solo permiten predecir el output de un modelo, sino que también aprender característcas en los datos, es decir, generar representaciones abstractas de los datos de entrada. 

En la práctica, los algoritmos de aprendizaje profundo son redes neuronales consistentes de neuronas y capas. Para diferenciarlos, es necesario observar la arquitectura y sus procesos de aprendizaje. Acá podemos distinguir a grandes rasgos las redes MLP o feedforward, convolucionales, Recurrentes y autoencoders. 

Antes de pasar a estudiar dichas arquitecturas, se discutirán algunos aspectos referentes al proceso de aprendizaje que en este tipo de algoritmos se relaciona estrechamente con los métodos de optimización a utilizar. 

Ya se aplicó el algoritmo de **descenso de gradiente estocástico SGD** en combinación con backpropagation. Este algoritmo puede ser modificado al introducir **momentum**. El algorimto SGD se puede resumir de la siguiente manera:

* Se sigue la dinámica de actualización $w \rightarrow w-\lambda \nabla(J(w))$, donde $\lambda$ se denota como *learning rate*. 

* Para añadir momentum, se puede añadir un *peso de actualización* de la forma:
$$
\triangle w \rightarrow \mu \cdot \triangle w-\lambda(\nabla J(w))
$$

Donde $\triangle w$ representa el vector de actualización añadido a $w$ en la iteración anterior, $\mu$ es un parámetro que determina la dependencia de nuevos valores para $w$ en función valores anteriores. $\mu \cdot \triangle w$ se denota como *momentum*. 

* Finalmente se crea una nueva regla de actualización mediante:
$$
w \rightarrow w+\Delta w
$$

**Ejercicio**

1. ¿Qué ventajas puede proporcionar añadir la componente de momentum?

2. Otro algoritmo útil es **Adam**, investigue sobre su funcionamiento y ventajas. [link](https://arxiv.org/abs/1412.6980)

### Redes Neuronales Convolucionales

Las redes neuronales convolucionales CNN (por sus siglas en ingles) fueron concebidas en el contexto de *visión computacional*. Aquí se busca obtener conocimiento por medio del manejo de representaciones interpretables de manera óptica por humanos (imagenes y videos por ejemplo). Tal proceso de obtención de información posee características bastante especificas y difíciles de codificar en una máquina. Por ejemplo, en el contexto de imagenes, se puede asumir que píxeles cercanos (bajo cierta métrica) se relacionan de mejor manera y que por tanto su aglomeración tiene sentido, por otra parte, el significado de una imagen puede depender fuertemente del contexto de ciertas estructuras o patrones abstractos difíciles de programar en un software. 

Una red neuronal múlticapa, en su forma más sencilla, considera a los datos de entrada como arreglos y a priori no saca provecho de la estructura de su input. En el ejemplo de las imagenes, tal red no puede distinguir vecinos posicionales (píxeles cercanos) pues recibe como input un arreglo unidimensional. En este contexto, nacen las CNN's, estas permiten sacar provecho de la estructura (especialmente en imágenes) haciendo posible mejorar la interacción entre neuronas cercanas. En problemas visuales, esto consiste en hacer que las neuronas procesen información originada en píxeles cercanos entre si, para lograr esto se hace uso de **capas convolucionales**.

Una capa convolucional consiste de un conjunto de filtros o *kernels*. Estos a la vez, consisten en un conjunto de pesos (a aprender), cada kernel es aplicado en *areas* de los datos de entrada. En la siguiente animación ([fuente](https://m-alcu.github.io/)) se puede observar un kernel de 3x3x3 aplicado sobre una imagen de 9x9x3, en ambos casos, el último índidce es 3 y corresponde a la cantidad de canales de color asociados a la imagen procesada.

![conv_net](https://m-alcu.github.io/assets/cntk103d_conv2d_final.gif)

Si la imagen es la capa input de la red, la acción del kernel consiste en asociar cada dato de entrada con un peso, en el ejemplo de la animación, el kernel aplicado posee 3x3x3 pesos que se *comparten* en cada región procesada de la imagen. El output de esta aplicación es una suma ponderada entre los píxeles de entrada y los pesos del filtro. Al recorrer toda la imagen, se tiene un nuevo conjunto de inputs sobre los cuales es posible operar con una función de activación u otra capa de neuronas (convolcional o simplemente lineal-conectada).

La idea intuitiva del kernel es de extraer alguna caracteística especifica de la imagen input (contornos, agrupaciones de colores, formas, etc...).  Gracias a que los pesos del kernel se mantienen mientras recorre la imagen, se logra una reducción en la cantidad de parámetros necesarios para entrenar una capa de la red. En algunas oportunidades, los kernels entrenados (conjuntos de pesos optimizados según alguna función de pérdida) pueden ser interpretables.

**Ejercicio**

1. Considere que se tiene una imagen input de dimensiones (*l*,*a*) y se aplica un filtro de tamaño (*fl*,*fa*). ¿ Cual es la dimensión del arreglo resultante? ¿Qué ocurre si se agrega profundidad de colores?

En términos generales, se dice que los filtros convolcionales operan sobre *volumenes de entrada*, esto permite abstraer el modelo a contextos más generales que problemas visuales. 

Una **Capa convolucional** consiste en un conjunto de filtros aplicados sobre un volumen de entrada, los volúmenes de salida de cada filtro conforman el output de la capa.

**Ejemplo**

Se construye un filtro y se aplica sobre una imagen. En primera instancia se define la función `conv` que toma una imagen y un filtro, luego opera el filtro sobre la imagen. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm


def conv(image, im_filter):
    # input dims
    height = image.shape[0]
    width = image.shape[1]
    
    # output dims
    im_c = np.zeros((height - len(im_filter) + 1, width - len(im_filter) + 1))

    # itera sobre la imagen
    for row in range(len(im_c)):
        for col in range(len(im_c[0])):

            # aplica el filtro
            for i in range(len(im_filter)):
                for j in range(len(im_filter[0])):

                    im_c[row, col] += image[row + i, col + j] * im_filter[i][j]

    # valores borde
    im_c[im_c > 255] = 255
    im_c[im_c < 0] = 0
    
    # obtiene imagenes
    plt.figure()
    plt.imshow(image, cmap=cm.Greys_r)
    plt.show()

    plt.imshow(im_c, cmap=cm.Greys_r)
    plt.show()
    
    return im_c

Se carga una imagen para operar, se transforma a escala de grises

In [ ]:
from PIL import Image
from urllib.request import urlopen

#Carga la imagen
image_rgb = np.asarray(
    Image.open(
        urlopen(
            'https://bloximages.newyork1.vip.townnews.com/stltoday.com/content/tncms/assets/v3/editorial/b/66/b66d25f2-a696-11df-89b6-00127992bc8b/4c64cfd425f2e.image.jpg'
        )).convert("RGB"))

# transforma a escala de grises
image_grayscale = np.mean(image_rgb, axis=2, dtype=np.uint)

Se define un filtro, este corresponde a un parche de 10x10 que en cada entrada tiene un peso de 1/100.

In [ ]:
#filtro blurr
blur = np.full([10, 10], 1. / 100)
blur

Al operar el filtro anterior en un parche de la imagen, se obtiene el promedio de las intensidades de los píxeles en el parche, el resultado es una imagen cuyos píxeles son promedios de parches de 10x10 en la imagen original, el resultado es el siguiente:

In [ ]:
volumen_output = conv(image_grayscale, blur)

In [ ]:
print('Dimensiones de la imagen input:', image_grayscale.shape)
print('Dimensiones de la imagen output:', volumen_output.shape)

Se pueden generar múltiples configuraciones de filtros sobre volumenes input. En el caso de la convolución implementada anteriormente, se tiene un volumen input de 415x620x1 a cual se aplica un único filtro de 10x10 generando un volumen output de 406x611x1. En el caso de operar sobre la imagen RGB asociada, el volumen input pasa a ser 415x620x3, en esta caso puede ser valido aplicar distintos filtros por canal generando un volumen output con incluso más canales de profundidad que el volumen input. También se puede aplicar el mismo filtro en cada canal, donde el output pasa a ser una combinación ponderada de los resultados a lo largo de los canales, en este caso, el volumen output tiene solo un canal de profundidad. En concreto, el esquema de capa convolucional es bastante flexible y no se resume solo al esquema visto como ejemplo.

En el ejemplo de convolución anterior, se asume que el filtro avanza un píxel por iteración. En general, ese no tiene por qué ser el caso, se puede configurar un filtro para avance múltiples pasos por iteración, el parametro que controla la longitud de estos *saltos* se denota como **stride**. En el ejemplo anterior se tiene un stride de 1.

Por lo general el stride es igual en cada dimensión del volumen input. Se espera que aumentar el stride de un filtro reduzca la dimensión del volumen output, en efecto, si se tiene un filtro con dimensiones $(f_w,f_h)$, se asume un stride de $s_w$ en el eje logitudinal y $s_h$ en el vertical. Al aplicar dicho filtro sobre una capa de $(w,h)$, se tiene que la capa output tendrá dimensión $((w-f_w)/s_w + 1, (h - f_h)/s_h + 1)$.

**Obs**: En este calculo se asume que los filtros son 2d (sin canales de profundidad). En este caso, para operar sobre volúmenes con profundidad se hace uso de una capa con tantos filtros como profundidad tenga el volumen a operar. 

Hasta ahora, las operaciones de convolución que se han revisado generan volúmenes output de menor dimensión. En la práctica este puede no ser deseable, para resolver esto, se introduce el concepto de **padding** o *relleno*. Esto consiste en agregar dimensiones artificiales en los contornos de un volumen input (ej: agregar píxeles con intensidad cero en los bordes de una imagen), generalmente estas dimensiones extras poseen valores nulos o promedios de sus vecinos. 

La manera más común de aplicar padding consiste en agregar dimensiones extras a la capa input, con tal de que la capa output posea la misma dimensión que la capa input sin modificar. Así, si se opera sobre una capa input de dimensión $(w,h)$ con padding $(p_w,p_h)$, por medio de un filtro con dimensión $(f_w,f_h)$ aplicando stride 2d con dimensiones $(s_w,s_h)$. El volumen de la capa output $(o_w,o_l)$ viene dado por:

$$
o_{w}=\frac{w+2 p_{w}-f_{w}}{s_{w}}+1
$$

$$
o_{h}=\frac{h+2 p_{h}-f_{h}}{s_{h}}+1
$$

### Capas de pooling

las capas de pooling corresponden a agregaciones de capas input en función de particiones regulares de estas. El proceso de *pooling* consiste en generar una grilla sobre una capa input cada celda de la grilla se denota como *campo receptivo*, sobre cada campo receptivo se aplica una operación de reducción / agregación (máximo, mínimo, promedio, etc...)

**Ejemplo**

Se define una capa input de 5x5x1:

In [ ]:
capa_input = np.random.rand(5,5)
capa_input

Sobre tal input se aplica una capa de pooling de 3x3 con stride 2 en ambas direcciones, la operación de reducción a aplicar es el promedio:

In [ ]:
from itertools import product

# Se genera la grilla sobre la capa input con stride 2 
grilla = (
    product(range(3), range(3)),
    product(range(3), range(2, 5)),
    product(range(2, 5), range(3)),
    product(range(2, 5), range(2, 5))
)

# Se recorre la capa input segun la grilla
capa_pool = np.zeros(4)
for i,g in enumerate(grilla):
    for grid_point in g:
        capa_pool[i] += capa_input[grid_point]
    
    capa_pool[i] /= 9
    
capa_pool = capa_pool.reshape([2,2])
capa_pool

Según el ejemplo anterior, se separa la capa input en sectores, cada sector es reducido a un escalar por medio del promedio de los elementos que lo conforman. 

**Ejercicio**

1. Encuentre una expresión para las dimensiones de una capa output sobre la cual se aplica una capa pooling.

Por lo general se alterna entre capas convolucionales y capas de pooling, de esta manera, las capas convolucionales detectan características, estás son abstraidas a sectores del volumen output por medio de agregaciones (capa pool). Aplicar capas convolucionales sobre las capas agregadas genera una busqueda de atributos sobre grupos de neuronas / píxeles más amplios, generando caraterísticas cada vez más abstractas. 

**Ejemplo**

Pytorch posee los módulos `torch.nn`, `torch.optim` yclases `Dataset`, `Dataloader`. Estas funcionalidades facilitan el trabajo con redes neuronales, se utilizarán estos objetos par entrenar una red neuronal convolucional sobre la base de digitos MNIST. Importamos las librerías iniciales y los datos a operar.

In [ ]:
import torchvision.datasets as datasets

mnist_train = datasets.MNIST(root='./data',
                             train=True,
                             download=True,
                             transform=None)

mnist_testset = datasets.MNIST(root='./data',
                               train=False,
                               download=True,
                               transform=None)

El conjunto de datos MNIST consiste en imagenes de 28x28 en escala de grises. Cada imagen está en formato `PIL` y representa un digito escrito a mano. El módulo `torchvision.datasets` incluye este dataset de manera libre (junto a otros conjuntos de datos emblematicos).

In [ ]:
mnist_train

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(mnist_train[0][0],cmap="gray")

Se define una rutina de preprocesamiento sobre el conjunto de datos

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import torch

X_tr = np.array([np.array(obs[0]) for obs in mnist_train])
y_tr = np.array([obs[1] for obs in mnist_train])

Se dividen los datos en entrenamiento y test

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(X_tr, y_tr, test_size = .2)

In [ ]:
x_train[0].shape

In [ ]:
from torch.utils.data import TensorDataset

train_ds = TensorDataset(torch.from_numpy(x_train), torch.from_numpy(y_train))
valid_ds = TensorDataset(torch.from_numpy(x_valid), torch.from_numpy(y_valid))

In [ ]:
# etiqueta
train_ds[0][1]

In [ ]:
# imagen
plt.imshow(train_ds[0][0], cmap = 'gray')

El objeto `TesorDataset` permite envolver tensores para generar un dataset compatible con otras estructuras de la librería. Para generar una dinamica de mini-batches, se utiliza la calse `DataLoader` :

In [ ]:
from torch.utils.data import DataLoader

bs = 64
train_dl = DataLoader(train_ds, batch_size=bs, shuffle=True)

En este caso, se crea un generador que entregara datos en mini batches de 64 elementos para operar en el proceso de entrenameinto. El parametro `shuffle` aleatoriza la generación de batches para evitar sobre ajuste. Se genera la estructura correspondiente para los conjuntos de validacion

In [ ]:
valid_dl = DataLoader(valid_ds, batch_size=bs * 2)

Se utiliza el módulo `torch.nn` de donde se adquieren las clases `Module` y `Parameter`. El procedimiento consiste en aplicar un esquema de herencia simple sobre `Module`, la clase resultante es un contendor de los pesos y sesgos de la red, además del método `.forward()` en el que se especifica el esquema de paso hacia adelante (procesmiento) que implementa la red. Un objeto de la clase  `Module` posee atributos como `.parameters`  y métodos como `.zero_grad()`. 

Definiremos una arquitectura de red convolucional con 2 capas de convolución: `conv1` con 32 filtros de 3x3, stride 1 y padding 0, `conv2` con 32 filtros (con 32 canales de entrada), filtros de 3x3, con stride 1 y padding 0. Posterior a la extracción de caractertísticas por convolución, se aplica una capa de pooling por máximo de tamaño 2x2 con stride 2. Luego de aumentar el campo receptivo con la capa de pooling, se vuelve a realizar un proceso de convolución con una capa de 2 filtros de 3x3 con padding cero y stride uno. Para finalizar el proceso se genera un arreglo unidimensional con las características extraidas, acá se aplica una sub estructura de percetrón multicapa con dos capas lineales completamente conectadas. El resultado de la red es un arreglo de 10 valores dados por la última capa lineal.

Para implementar dichas capas convolucionales, se utiliza el objeto `nn.Conv2d`, la capa de pooling se implementa por medio de `nn.MaxPool2d`, la capa de transformación unidimensional por medio de `nn.Flatten`, finalmente las capas lineales por medio de `nn.Linear`.

In [ ]:
from torch import nn

class Mnist_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0)
        self.max_pool = nn.MaxPool2d(2, stride=2)
        self.conv3 = nn.Conv2d(32,2, kernel_size=3)
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(200,50)
        self.linear2 = nn.Linear(50,10)

La arquitectura de la red se define en el *forward pass* esto se hace anulando el método forward. En este caso se hace uso del  módulo `functional` de `torch.nn`. La estructura a usar será, utilizar funciones de activación ReLu sobre los outputs de cada capa convolucional, para luego aplicar una capa de pooling por promedio 

In [ ]:
import torch.nn.functional as F
from torch import nn

class Mnist_CNN(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0)
        self.max_pool = nn.MaxPool2d(2, stride=2)
        self.conv3 = nn.Conv2d(32,2, kernel_size=3)
        self.flatten = nn.Flatten()
        self.linear1 = nn.Linear(200,50)
        self.linear2 = nn.Linear(50,10)
    
    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = F.relu(self.conv1(xb.float()))
        xb = F.relu(self.conv2(xb))
        xb = self.max_pool(xb)
        xb = F.relu(self.conv3(xb))
        xb = self.flatten(xb)
        xb = F.relu(self.linear1(xb))
        xb = F.relu(self.linear2(xb))
        
        return xb

Para entrenar el modelo se utiliza SGD con momentum, para hacer uso de este optimizador usamos el módulo `optim`

In [ ]:
from torch import optim

lr = 0.1
model = Mnist_CNN()
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

A continuación se define una rutina de entrenamiento, esta se implementa como la función `fit`. Para ello se define la función de pérdida:

In [ ]:
import torch
loss_func = torch.nn.CrossEntropyLoss()

Luego se define la función de perdida por batch:

In [ ]:
def loss_batch(model, loss_func, xb, yb, opt=None):
    loss = loss_func(model(xb), yb)

    if opt is not None:
        loss.backward()
        opt.step()
        opt.zero_grad()

    return loss.item(), len(xb)

Para definir el proceso de entrenamiento se define la siguiente función auxiliar:

In [ ]:
import numpy as np 

def register(res_list):
    '''Obtiene la perdida promedio y su desviacion estandar en batches.'''
    
    losses, nums = zip(*res_list)
    
    N = np.sum(nums)
    loss_mean = np.sum(np.multiply(losses, nums))/N
    loss_std = np.sqrt(np.sum(np.multiply((losses-loss_mean)**2, nums))/(N-1))
    
    return loss_mean, loss_std

Finalmente se define el proceso de entrenamiento

In [ ]:
import pandas as pd

def fit(epochs,
        model,
        loss_func,
        opt,
        train_dl,
        valid_dl,
        metric=None,
        only_print=True,
        print_leap = 1):
    
    '''Fit - Entrena una red neuronal.
    
    Entrena una red neronal dada por model, para esto recibe como argumento
    una funcion de perdida, un optimizador, DataLoaders para entrenamiento 
    y validacion. Muestra en pantalla los errores de validacion y entrega
    como resultado una lista con los resultados del entrenamiento.

    Parameters
    ----------
    epochs: int
        Numero de epocas a entrenar.
            
    model: Mnist_CNN
        Red convolucional tipo Mnist_CNN a entrenar.
           
    loss_func: function
        Funcion de perdida como criterio de la red.
        
    opt: torch.optim Object
        Opimizador de la red.
        
    train_dl: torch.utils.data.dataloader.DataLoader
        DataLoader del conjunto de entrenamiento.
        
    valid_dl:torch.utils.data.dataloader.DataLoader
        DataLoader del conjunto de validacion.
    
    metric: function
        Metrica de rendimiento propia. El modelo se entrena con la funcion de 
        perdida pero se pueden almacenar resultados de prediccion utilizando
        esta metrica.
    
    only_print: bool 
        Indica si se quiere retornar un registro del proceso o solo imprimir 
        en pantalla.
        
    print_leap: int
        Cada cuanntas epocas se desea mostrar resultados en pantalla.
    
    Returns
    -------
    learning_data: Panda DataFrame
        retorna un DataFrame con columnas:
        
        epoch: numero de epocas.
        train_loss: perdida promedio en entrenamiento.
        train_std: desviacion de la perdida en entenamiento.
        val_loss: perdida promedio en validacion.
        val_std: desviacion de la perdida en validacion.
    '''
    if metric is None:
        metric = loss_func

    learning_data = pd.DataFrame(
        columns=['epoch', 'train_mean', 'train_std', 'val_mean', 'val_std'])

    for epoch in range(epochs):

        # Entrenamiento -------------------------------------------------------
        train_res = []
        model.train()

        for xb, yb in train_dl:
            # Para entrenar se usa la funcion de perdida
            loss_batch(model, loss_func, xb, yb, opt)

            # Para almacenar se puede usar una metrica
            train_res.append(loss_batch(model, metric, xb, yb))

        # Validacion ----------------------------------------------------------
        # Para evaluar se puede utilizar un metrica de rendimiento
        model.eval()

        with torch.no_grad():
            val_res = [
                loss_batch(model, metric, xb, yb) for xb, yb in valid_dl
            ]

        val_loss, val_std = register(val_res)
        train_loss, train_std = register(train_res)
        
        if epoch % print_leap == 0:
            print('Epoca:', epoch, '- val:', val_loss, '- train:', train_loss)

        learning_data = learning_data.append(
            {
                'epoch': epoch,
                'train_mean': train_loss,
                'train_std': train_std,
                'val_mean': val_loss,
                'val_std': val_std
            },
            ignore_index=True)

    if only_print:
        print('Proceso terminado')
    else:
        return learning_data

La función `get_data` permite obtener conjuntos de validacion y entrenamiento, además optimiza el proceso de carga de manera paralela (CPU) utilizando `n_workers` como parámetro.

In [ ]:
n_cores = 4
def get_data(train_ds, valid_ds, bs):
    return (
        DataLoader(train_ds,
                   batch_size=bs,
                   shuffle=True,
                   num_workers=n_cores),
        DataLoader(valid_ds,
                   batch_size=bs * 2,
                   num_workers=n_cores),
    )

In [ ]:
epochs = 2
bs = 3000

# Se generan los generadores de batches
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)

# Declara el modelo y optimizador
lr = 0.1

model = Mnist_CNN()
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

Finalmente se entrena el modelo para 2 épcoas

In [ ]:
%%time
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

**Ejercicio**

1. Implemente la red anterior utilizando la clase `Sequential` del módulo `torch.nn`. 

Se traspasa el modelo anterior a la GPU y se estudia diferencia en tiempo de ejecución. Para ello, observa si la unidad de procesamiento gráfico esta disponible

In [ ]:
print('GPU disponible:' , torch.cuda.is_available())

como ya se vio anteriormente, se puede utilizar el método `.to()` para enviar tensores a la GPU, se puede realizar la misma operación sobre nuestra red convolucional. En primera instancia se almacena la dirección de la GPU a trabajar (si hay una disponible)

In [ ]:
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

print('Dispositivo dispoble:', dev)

movemos el modelo a la GPU, esto implica mover el tensor asociado a sus pesos, por tal motivo, el optimizador debe ser inicializado nuevamente

In [ ]:
model.to(dev)
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

Para entrenar el modelo es necesario ingresar los datos a procesar a la GPU, esto se puede implementar en la funcion `.fit()` anterior. Otra forma de hacer este cambio es modificando la clase `DataLoader` para que preprocese los datos antes de producir los generadores correspondientes, para ello se implementa la clase `ExtendedLoader` que toma un objeto `DataLoader` y permite modificarlo entregando un generador transformado:

In [ ]:
class ExtendedLoader:
    '''Extiende un objeto DataLoader.'''
    
    def __init__(self, data_loader, transf):
        self.data_loader = data_loader
        self.transf = transf
    
    # Emula los metodos necesarios de un DataLoader
    def __len__(self):
        return len(self.data_loader)

    def __iter__(self):
        # Genera un iterable para aplicar un ciclo for
        batches = iter(self.data_loader)
        
        '''
        Se modifican los objetos generados por el data loader
        y se obtiene un nuevo generador (observe el comando yield).
        '''
        for b in batches:
            yield (self.transf(*b))

Se define la transformación a realizar para extender un `DataLoader`

In [ ]:
def transformacion(x,y, dev = dev):
    return x.view(-1, 1, 28, 28).to(dev), y.to(dev)

Finalmente se entrena el modelo utilizando las estructuras generadas

In [ ]:
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)

train_dl = ExtendedLoader(train_dl, transformacion)
valid_dl = ExtendedLoader(valid_dl, transformacion)

In [ ]:
%%time
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

se observa una reducción de tiempo del orden de 8 veces el tiempo en CPU (depende de la máquina donde se ejecuta). Se procede a estudiar el comportamiento del modelo entrenado bajo GPU, para eso se generan **curvas de aprendizaje**. Estas consisten en visualizaciones sobre el rendimiento de un modelo, tanto en entrenamiento como en test/validación. 

Este tipo de herramientas son ampliamente utilizadas para diagnosticar el progreso del aprendizaje de manera secuencial. Utilizando la información que estas curvas es posible tener una idea del ajuste (sobre o bajo) de un modelo. En general, puede construirse una curva de aprendizaje grafícando una métrica de rendimiento de algún modelo en función de alguna cantidad ordenada que afecte en los valores de esta (ej: error de un modelo de regresión lineal en validación versus cantidad de observaciones usadas en entrenamiento). En este caso se comparan la cantidad de épocas contra la exactitud (accuracy) esto está implementado en la función `fit`. Los resultados se almacenan en el `DataFrame` `learning_data`.

In [ ]:
def accuracy(y_hat_b,yb):
    
    preds = torch.argmax(torch.softmax(y_hat_b,dim = 1),dim=1)
    counts = (preds == yb)*1.0
    
    return torch.mean(counts)

Una vez definida la métrica a registrar, se procede a definir el modelo

In [ ]:
torch.manual_seed(6202)

model = Mnist_CNN()
lr = .1
mo = .8

model.to(dev)
opt = optim.SGD(model.parameters(), lr=lr, momentum=mo)

In [ ]:
%%time
bs = 3000

epochs = 10
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)

train_dl = ExtendedLoader(train_dl, transformacion)
valid_dl = ExtendedLoader(valid_dl, transformacion)

leap = 1

learning_data = fit(epochs,
                    model,
                    loss_func,
                    opt,
                    train_dl,
                    valid_dl,
                    metric=accuracy,
                    only_print=False,
                    print_leap=leap)

Se programa una función para obtener las curvas de aprendizaje recolectadas en `fit`

In [ ]:
import matplotlib.pyplot as plt

def show_learning_curves(learning_data, leaps = 1):
    '''Genera curvas de aprendizaje dado data dataframe resultado de fit().'''
    
    fig, ax = plt.subplots(figsize=[10, 7])
    ax.grid()


    epcs = learning_data['epoch'][::leaps]
    val_loss_lower = (learning_data['val_mean'] -
                      learning_data['val_std'])[::leaps]

    val_loss_upper = (learning_data['val_mean'] +
                      learning_data['val_std'])[::leaps]

    train_loss_lower = (learning_data['train_mean'] -
                        learning_data['train_std'])[::leaps]
    train_loss_upper = (learning_data['train_mean'] +
                        learning_data['train_std'])[::leaps]

    ax.plot(epcs,
            learning_data['val_mean'][::leaps],
            'o--',
            color="g",
            label="Validation")

    ax.fill_between(epcs, val_loss_lower, val_loss_upper, alpha=0.1, color='g')

    ax.plot(epcs,
            learning_data['train_mean'][::leaps],
            'o--',
            color="r",
            label="Train")

    ax.fill_between(epcs, train_loss_lower, train_loss_upper, alpha=0.1, color='r')

    ax.set_title('Curva de Aprendizaje', fontsize=25)
    ax.set_xlabel('Epocas', fontsize=15)
    ax.set_ylabel('Accuracy', fontsize=15)
    ax.legend()

Se observan las curvas de aprendizaje 

In [ ]:
show_learning_curves(learning_data)

Lo que esta curva indica es que la dinámica de entrenamiento es incapaz de capturar patrones en los datos. Esto se observa, pues tanto las métricas de validación como de entrenamiento son bajas, además de se prácticamente constantes a lo largo de cada época. Esto quiere decir que nos encontramos frente a un modelo con *underfit*. Para solucionar ese problema es posible aumentar el tiempo de entrenamiento o cambiar el esquema de optimización para acelerar la búsqueda de parámetros. 


En este caso se utilizará el algoritmo **AdaDelta**. 

### Interludio: AdaDelta

Este algoritmo es una variante al método de descenso de gradiente estocástico. La diferencia que se incluye en este caso es la posibilidad de ajustar hiperparámetros de manera adaptativa. 

En primer lugar, se busca resolver el siguiente problema:
$$
\min _{w} F(x):=\sum_{i=1}^{n} f_{i}(x)
$$

Acá se busca obtener una dinámica de descenso de gradiente en la cual se calcula el *learning rate* sistemáticamente, de manera que se garantice convergencia rápida. Para ello, se calcula un *learning rate* por dimensión basándose en las condiciones de primer orden de optimalidad. En primer lugar se define:

$$
\mathbb{E}\left[g^{2}\right]_{t}=\rho \mathbb{E}\left[g^{2}\right]_{t-1}+(1-\rho)\left(\nabla f_{i_{i}}\left(x_{t}\right)\right)^{2}
$$

con la condición inicial $\mathbb{E}[g^2]_0 = 0$. Acá, $\left(\nabla f(x_t)\right)^2 \in \mathbb{R}^p$ y corresponde al vector que almacena los gradientes, elevados al cuadrado. utilizando $\mathbb{E}[g^2]_t$ se construye una regla de descenso de gradiente según:

\begin{align}
x_{t+1}&= x_{t}+\Delta x_{t} \\
&=x_{t}-\frac{\eta}{\mathrm{RMS}[g]_{t}} \odot \nabla f_{i_{t}}\left(x_{t}\right)\\
&=x_{t}-\frac{\eta}{\sqrt{\mathbb{E}\left[g^{2}\right]_{t}+\varepsilon}} \odot \nabla f_{i_{t}}\left(x_{t}\right)
\end{align}

donde $\text{RMS}[\cdot]$ es la media cuadrática bajo raíz.  Al definir $\mathbb{E}\left[\Delta x^{2}\right]_{t}=\rho \mathbb{E}\left[\Delta x^{2}\right]_{t-1}+(1-\rho) \Delta x_{t}^{2}$, con condición inicial  $\mathbb{E}[\Delta x^2]_0 = 0$. Se define el paso AdaDelta cambiando $\eta$ por $\mathrm{RMS}[\Delta x]_{t-1}$, dando origen a:

\begin{align}
x_{t+1}&=x_{t}-\frac{\operatorname{RMS}[\Delta x]_{t-1}}{\operatorname{RMS}[g]_{t}} \odot \nabla f_{i_{t}}\left(x_{t}\right)\\
&=x_{t}-\frac{\sqrt{\mathbb{E}\left[\Delta x^{2}\right]_{t}+\varepsilon}}{\sqrt{\mathbb{E}\left[g^{2}\right]_{t}+\varepsilon}} \odot \nabla f_{i_{t}}\left(x_{t}\right)
\end{align}

Para aplicar este algoritmo se puede hacer uso del objeto `Adadelta` del módulo `optim`. Se procede a entrenar la red anterior utilizando un optimizador de Adadelta.

**Obs**: Se puede acceder a la formulación del algoritmo por medio del siguiente [recurso](https://arxiv.org/abs/1212.5701).

In [ ]:
torch.manual_seed(6202)
model = Mnist_CNN()
model.to(dev)

opt = optim.Adadelta(model.parameters())

Se procede a entrenar el modelo y a observar las curvas de aprendizaje

In [ ]:
bs = 3000

epochs = 20
train_dl, valid_dl = get_data(train_ds, valid_ds, bs)

train_dl = ExtendedLoader(train_dl, transformacion)
valid_dl = ExtendedLoader(valid_dl, transformacion)

leap = 2

learning_data = fit(epochs,
                    model,
                    loss_func,
                    opt,
                    train_dl,
                    valid_dl,
                    metric=accuracy,
                    only_print=False,
                    print_leap=leap)

In [ ]:
show_learning_curves(learning_data)

la curva de aprendizaje muestra en este caso que se logra un crecimiento sostenido hasta las 6 épocas, luego de eso los parámetros de la red comienzan a arrojar resultados similares, no se aprecia sobreajuste pues la curva de validación es consistente con la de entrenamiento pues se mantiene en rango similares. Lo anterior nos indica que el modelo es capaz de aprender la información contenida en los datos, es posible seguir entrenado la red para ver si es posible mejorar las métricas de rendimiento, esto se debe juzgar en función de la disponibilidad de tiempo y recursos para enfrentar el problema. Para finalizar, se hace un estudio del rendimiento en el conjunto de test inicialmente separado.

In [ ]:
mnist_testset

Se obtiene los datos y etiquetas para procesar

In [ ]:
X = mnist_testset.data
y = mnist_testset.targets.numpy()

se traspasa el modelo a cpu para hacer al evaluación

In [ ]:
model.to('cpu')

se obtienen las predicciones

In [ ]:
model.eval()
with torch.no_grad():
    y_hat = torch.softmax(model(X),dim = 1)
    y_hat = torch.argmax(y_hat,dim=1).numpy()

finalmente se genera un reporte con los resultados en el conjunto de test

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y,y_hat))

los resultados muestran que el modelo es capaz de generalizar fuera del conjunto de entrenamiento.

**Ejercicio**

1. Es posible mejorar el rendimiento y generalización en algoritmos de aprendizaje profundo al realizar preprocesamientos como normalizaciones , reducción de dimensionalidad, estandarización, entre otras. Aplique un esquema de preprocesamiento adecuado y observe como este afecta los resultados de la red (ej: normalice los datos a intensidad en (0,1)).

Las curvas de aprendizaje anteriores muestran un buen comportamiento en cuanto a la capacidad de generalización del modelo. Esto se confirma con los resultados obtenidos en el conjunto de test. Sin embargo, este no siempre es el caso, pudiendose caer modelos sobreajustados. 

Para lidiar con este tipo de situaciones, se pueden aplicar **técnicas de regularización**. Dentro de estas herramientas encontramos:

**Decaimiento en los pesos**

Consiste en una generalización de la regularización L2 en redes neuronales. Consideremos por ejemplo una función de pérdida $L_0(w,y,\hat{y})$, la versión regularizada de esta función de pérdida corresponde a:

$$
L(w,y,\hat{y}) = L_0(w,y,\hat{y}) + \frac{\lambda}{2 n} \sum_{i} w_i^{2}
$$

Al derivar con respecto a los parámtros $w$ del modelo se obtiene:
$$
\frac{\partial L}{\partial w_i}=\frac{\partial L_{0}}{\partial w_i}+\frac{\lambda}{n} w_i
$$

con lo que, al aplicar un esquema de descenso de gradiente, se obtiene un esquema de actualización de pesos dado por:

$$
w  \rightarrow \left(1-\frac{\eta \lambda}{n}\right) w-\eta \nabla L_{0}
$$

El esquema de decaimiento de pesos utiliza lo anteriormente diseñando para generar una **nueva** dinámica de actualización de pesos dada por: 

$$
\mathrm{w} \rightarrow (1-\lambda) w-\eta \Delta J
$$

En este caso, $J$ representa la función de pérdida de la red a entrenar, $\lambda$ el parámetro de decaimiento de pesos y $\eta$ la tasa de aprendizaje (*learning rate*). 

**Ejercicio**

1. ¿Como afecta el parámetro $\lambda$ a la dinámica de entrenamiento? 

Los optimizadores de Pytroch permite utilizar esta técnica de regularización por medio del parámetro `weight_decay`.

**Ejemplo**

Para la red anterior, es posible usar un optimizador regularizado por decaimiento de pesos al definir:

In [ ]:
w_d = 0.5
opt = optim.Adadelta(model.parameters(), weight_decay=w_d)

**Dropout**

Otra tpecnica de regularización es el *dropout*. Esta técnica consiste en eliminar neuronas output en algunas capas. El algoritmo consiste en seleccionar neuronas al azar y quitar sus outputs de modelos. Estas neuronas no son completamente eliminadas del modelo, en una siguiente iteración (mini-batch) se seleccionan otras neuronas para ser apagadas. En general, en cada mini-batch, cada neurona tiene una probabilidad $p$ se ser seleccionada y "apagada".  La razón de este método consiste en asegurar que todas las neuronas del modelo aprendan carácteristicas del problema, evitando replicar el comportamiento de otras neuronas de capas anteriores. 

Esta técnica es consistente con cualquier tipo de capa (convolucional, poolling, completamente contectadas, ...). En Pytorch se tiene acceso al método *Dropout* por medio de `nn.Dropout`. Los objetos de esta clase son operados como con cualquier otra capa de la red (se comportan como una capa más de la arquitectura).

**Ejercicio**

1. Agregue una capa Dropout a la arquitectura de red generada para la base MNIST.

**Data Augmentation**

Corresponde a una de las técnicas más eficientes de regularización. Consiste en aplicar transformaciones sobre los conjuntos de entrenamiento, estas transformaciones *conservan la clase* de las observaciones, es decir, no modifican el output asociado a ella. La intención de producir estas transformaciones es de entrenar un modelo capaz de capturar invarianzas en el fenómeno modelado. 

**Ejemplo**

En el campo de procesamiento de imagenes, las técnicas de regularización son ampliamente utilizadas. Las más comunes son rotaciones, reflexiones horizontales/verticales, acercamientos, reescalamientos, cortes, traslaciones, transformaciones de brillo y contraste, entre otras.  

Las imagenes de la base MNIST puede ser procesadas utilizando la librería `PIL`, la ventaja de esta librería es que trabaja de manera nativa con Pytorch por medio del modulo `torchvision`. 

Se importan los módulos a trabajar y se selecciona una imagen ejemplo del dataset MNIST.

In [ ]:
from torchvision import transforms
from PIL import Image

mnist_example = mnist_train.data[2]

la manera de transformar un tensor en una imagen de PIL es por medio de la función `ToPILImage` del módulo `torchvision.transforms`.

In [ ]:
img = transforms.ToPILImage()(mnist_example)
img

las imagenes de PIL son compatibles con `matplotlib`, es posible por ejemplo, visualizar una imagen pil de manera nativa por medio de:

In [ ]:
plt.imshow(img, cmap = 'gray')

Para aplicar una rotacion a la imagen anterior, se puede utilizar el método `rotate`, este recibe grados como argumento.

In [ ]:
plt.imshow(img.rotate(45) , cmap = 'gray')

para cortar la imagen es posible utilizar el método `crop`, este recibe una tupla de 4 coordenadas, dada por:
```
(p_sup_x, p_sup_y, p_inf_x, p_inf_y)
```

donde `p_sup_x,p_sup_y` corresponde al vértice superior izquierdo de un rectangulo, como coordenadas de pixeles en la imagen a cortar. Por otra parte, ` p_inf_x, p_inf_y` corresponde al vértice inferior derecho de un rectangulo. El rectangulo representado por las corrdenadas entregadas será la imagen entregada por `crop`.

In [ ]:
plt.imshow(img.crop((5,5,23,23)), cmap = 'gray')

**Ejercicio**

1. Se puede cambiar el tamaño de imagen utilizando el método `.resize()`. ¿Qué ocurre si se entrega una tupla con dimensiones superiores a la de la imagen a procesar?

**Normalización de Batches**

Este procedimiento consiste en normalizar los outputs de una capa oculta para cada mini-batch, esta normalización mantiene un valor de activación promedio igual a 0 en la capa normalizada y un desviación estándar igual a 1. Se puede utilizar este tipo de normalización con cualquier tipo de capa oculta. 

Por lo general, las redes con capas normalizdas son más rápidas de entrenar y pueden utilizar tasas de aprendizaje más grandes. En Pytorch es posible acceder a este tipo de normalización por medio de `nn.BatchNorm1d` para capas de dimensión (N,L) donde N es la cantidad de elementos en el batch, L es la cantidad de neuronas de la capa a operar (en este caso estamos frente a una capa lineal de L outputs que será normalizada).`nn.BatchNorm1d` también opera sobre capas con output de la forma (N,C,L), donde en este caso C representa la cantidad de canales de output. Si se quiere normalizar una capa con output del tipo (N,C,H,W) se puede utilizar `nn.BatchNorm2d` en este caso, H representa el alto y W el ancho del volumen con C canales de profundidad y N elementos en batch. 

En todos estos casos, las dimensiones de output son preservadas.

**Ejemplo** 

Se agregan 2 capas de batch normalization a la red antes generada, la nueva red se denota como `MnistCnnBatchNorm`. 

La primera capa se denota como `batch_norm_1`, debido a que la dimensión luego de la primera capa convolucional es del tipo (N,32,26,26), se inicializa  `batch_norm_1` con el valor `num_features = 32` pues este parámetro debe coincidir con C en inputs de la forma (N,C,H,W).

Para `batch_norm_2` se utiliza `num_features = 32` pues aquí se opera sobre outputs de la forma (N,200) es decir, del tipo (N,L). En tal caso, el parámetro `num_features` debe coincidir con L. 

**Obs:** Si se opera esta nueva red con batches de tamaño 1 habrá un error pues la normalización es calculada según $y = (x - \bar{x}) / (std(x) + eps)$. Acá $\bar{x} = x$  por lo que todos los pesos pasarían a ser cero. Para poder evaluar la red en ejemplos singulares (batches de tamaño 1), se debe poner la red en modo de evaluación utilizando el método `.eval()`.

In [ ]:
class MnistCnnBatchNorm(nn.Module):
    
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0)
        self.conv2 = nn.Conv2d(32, 32, kernel_size=3, stride=1, padding=0)
        
        self.batch_norm_1 = nn.BatchNorm2d(num_features=32)
        
        self.max_pool = nn.MaxPool2d(2, stride=2)
        self.conv3 = nn.Conv2d(32,2, kernel_size=3)
        self.flatten = nn.Flatten()
        
        self.batch_norm_2 = nn.BatchNorm1d(num_features=200)
        
        self.linear1 = nn.Linear(200,50)
        self.linear2 = nn.Linear(50,10)
    
    def forward(self, xb):
        xb = xb.view(-1, 1, 28, 28)
        xb = F.relu(self.conv1(xb.float()))
        xb = self.batch_norm_1(xb)
        xb = F.relu(self.conv2(xb))
        xb = self.max_pool(xb)
        xb = F.relu(self.conv3(xb))
        xb = self.flatten(xb)
        xb = self.batch_norm_2(xb)
        xb = F.relu(self.linear1(xb))
        xb = F.relu(self.linear2(xb))
        
        return xb

se comprueba que la red preserva las dimensiones al operar sobre la imagen de prueba

In [ ]:
mod_norm = MnistCnnBatchNorm()
mod_norm.eval()
mod_norm(mnist_example).shape

Lo anterior confirma que la red fue bien configurada en cuanto a las dimensiones de sus capas.

**Ejercicio**

1. Compare los tiempos de entrenamiento y métricas de rendimiento entre esta versión de la red convolucional y la red sin normalización por batches. 

## Transferencia de Aprendizaje

Las redes estudiadas hasta el momento han sido diseñadas para consumir pocos recursos computaciones. En algunos problemas, debido a la información disponible y a la complejidad de problema modelado, se hace necesario utilizar más recursos y generar redes más complejas. Estas redes pueden no ser entrenadas con los recursos que se poseen. Una solución al problema de acceso a recursos consiste en utilizar redes pre-entrenadas, esta técnica se conoce como **transferencia de aprendizaje** o *transfer learning*. 

En términos generales, la transferencia de aprendizaje consiste en utilizar un modelo de aprendizaje de máquinas entrenado en un conjutno de datos, para que haga predicciones sobre un nuevo conjunto de datos relacionado, pero no exactamente igual al inicial. 

Por ejemplo se puede utilizar un red entrenada en [*ImageNet*](http://image-net.org/) para hacer detección de obejetos en un entorno particular. 

**Obs**: El conjunto de datos *ImageNet* corresponde a una recopilación de imagenes etiquetadas con más de 20.000 categorias y más de 14 millones de observaciones. Trabajar el problema de clasificación asociado a este conjunto de datos se conoce difícil y de altos requerimientos computacionales. Actualmente se enfrenta tal problema con redes del orden de 480 millones de parámetros logrando una exactitud del orden del 98%. 

Dentro de las maneras más comunes de aplicar transferencia de aprendizaje a redes neuronales se encuentran: 

1. Utilizar una red pre-entrenada como un extractor de características, sobre tal arquitectura se pueden agregar nuevas capas de neuronas sobre las cuales se aplica un esquema de entrenamiento. Acá se "congelan" los pesos de red pre-entrenada y sólo se actualizan los pesos extra al propagar hacia atrás. 

2. Entrenar la red completamente (*fine tunning*). En este contexto, se entrena la red completamente (se pueden agregar capas si se desea). Se pueden bloquear algunas capas y entrenar sobre otras.

Como idea general, se dice que las capas iniciales de una red son aquellas que captan caracterísitcas más generales, mientras que las capas más profundas aprenden características abstractas y relacionadas al problema especifico que ataca la red. Por tal motivo, en el caso 2, se recomienda congelar los pesos de las primeras capas y entrenar sobre el resto de la red. 

**Ejemplo**

Se accede a la base de datos CIFAR-10, la cual posee 60.000 imagenes de 32x32 a color, agrupadas en 10 clases. Siguiendo el procedimiento anterior, cargamos la base desde el módulo `datasets` de `torchvision`.

In [ ]:
import torch
from torch import nn, optim

from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms, datasets

import pandas as pd 
import numpy as np 

cifar_train = datasets.CIFAR10(root='./data',
                             train=True,
                             download=True)

cifar_test = datasets.CIFAR10(root='./data',
                               train=False,
                               download=True)

In [ ]:
cifar_train

Se observan una de las imagenes del dataset

In [ ]:
# Un camion
import matplotlib.pyplot as plt
plt.imshow(cifar_train.data[30])

se busca utilizar la red [ResNet-18](https://arxiv.org/abs/1512.03385), esta se clasifica como una *red residual*. Una red residual consiste en una arquitectura *no directamente secuencial* pues genera una conexión paralela entre capas no adyacentes. La idea es una red residual es replicar el input de la red en capas ocultas, para eso genera una conexión paralela y una capa oculta de igual dimensión. La conexión entre ambas capas corresponde a la suma vectorial de sus outputs. Se puede tener por ejemplo un **bloque residual** de la forma:

```
x --> capa_1 --> capa_2 --> capa_3 --> f(x) + x --> ...
```

Que se puede representar por:

```
x --> f_1(x) --> f_2(f_1(x)) --> f_3(f_2(f_1(x)) + x --> ...
```

Se observa que el input `x` se vuelve a introducir al final del bloque residual. Esta dinámica se puede combinar con capas de todo tipo, solo basta generar bloques con capas output de igual dimensión al input. 

**Ejercicio**

1. Investigue sobre las capas *Bottle Neck* y su relación las redes residuales

Continuando con el ejemplo, dado que *ResNet-18* está entrenada sobre la red *ImageNet*, es necesario transformar los inputs de CIFAR-10 para que posean dimensión 224x224, además se deben estandarizar los píxeles de CIFAR-10 utilizando la media y desviación estándar. 

Para implementar una *pipeline* de transformaciones se hace uso de la Clase `Compose` del módulo `transforms`. Esta clase actúa como una pipeline de Scikit-learn y acepta como inputs objetos del módulo `transforms`.

In [ ]:
#Se trabaja con la GPU para mayor eficiencia:
dev = torch.device("cuda") if torch.cuda.is_available() else torch.device(
    "cpu")

train_data_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

En el caso anterior se agregan reflejos horizontales y verticales con el fin de aplicar aumentación de datos sobre el conjunto de entrenamiento y poder regularizar la red a implementar. 

Se procede a construir un conjunto de entrenamiento utilizando la composición de transformaciones anteriores, se crea además un objeto `DataLoader` para generar batches.

**Obs:** las transformaciones diseñadas pueden ser aplicadas al conjunto de imagenes ya cargado, lo cual es una práctica habitual. Sin embargo, dado que Pytorch entrega soporte al conjunto CIFAR10 en su módulo `torchvision.datasets`, es posible cargar los datos directamente entregando un objeto de transformaciones.

In [ ]:
cifar_train = datasets.CIFAR10(root='./data',
                             train=True,
                             download=True,
                             transform=train_data_transform)

Se genera un conjunto de transformaciones para el conjunto de test. En este caso, no es necesario realizar las operaciones de aumentación.

In [ ]:
test_data_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
cifar_test = datasets.CIFAR10(root='./data',
                               train=False,
                               download=True,
                               transform=test_data_transform)

se carga ResNet-18:

In [ ]:
resnet = torchvision.models.resnet18(pretrained=True)

la función `summary` del módulo `torchsummary` permite visualizar la estructura de la red anterior de manera estilizada. Para utilizar dicha función, debemos proporcionar el modelo y las dimensiones del input. 

In [ ]:
from torchsummary import summary
summary(resnet.cuda(),(3,224,244))

al hacer un `print` del modelo tenemos acceso a un esquema similar:

In [ ]:
print(resnet)

Acá se observa que la última capa de la red tiene dimensión 1000 de salida y su nombre es `fc`. Como las capas son atributos de las redes podemos acceder a dicha capa por medio de `resnet.fc`. Con esta información, se crea una arquitectura consistente en 2 capas lineales (o *fully connected*) que reducen gradualmente la dimensión desde 1000 hasta 200, desde 200 hasta 10. La primera capa lineal será la última capa de la red anterior, mientras que la segunda se define usando `nn.Linear`. Esto se implementa utilizando el objeto `nn.Sequential` puesto que la estructura *forward* consiste en aplicar la red como extractor y luego (de manera secuencia) recorrer las ultimas dos capas con parámetros sujetos a aprendizaje.

Lo anterior pretende realizar transferencia de aprendizaje utilizando la red pre-entrenada como una extractora de características. Para ello, se desactivan los gradientes asociados a sus parámetros:

In [ ]:
for param in resnet.parameters():
    param.requires_grad = False

Se pasa a construir la arquitectura de red sobre el extractor de características

In [ ]:
num_features = resnet.fc.in_features

resnet.fc = nn.Linear(num_features, 200)
resnet.to(dev)

upper_net = nn.Linear(200, 10)
upper_net.to(dev)

finalmente se obtiene la arquitectura 

In [ ]:
from collections import OrderedDict
steps = OrderedDict([('feature_extractor', resnet), 
                     ('upper_net', upper_net)])

transfered_net = nn.Sequential(steps)
transfered_net.to(torch.float16)
transfered_net.to(dev)

como función de perdida se utiliza la entropia cruzada

In [ ]:
loss_func = nn.CrossEntropyLoss()

Como optimizador se usa *Adam*

In [ ]:
opt = optim.Adam(transfered_net.parameters(), lr = 0.1)

se genera un conjunto de validación y uno de entrenamiento (sin usar Scikit-learn).

In [ ]:
valid_prop = .2
train_size = int((1 - valid_prop) * len(cifar_train))
test_size = len(cifar_train) - train_size

train_ds, valid_ds = torch.utils.data.random_split(cifar_train,
                                                   [train_size, test_size])

se utiliza la clase `ExtendedLoader` para crear un `DataLoader` extendido, capaz de cargar los batches en GPU al momento de generarlos. Para ello se define la transformación:

In [ ]:
def transformacion_2(x,y,dev=dev):
    x = x.to(torch.float16)
    return x.to(dev), y.to(dev)

busca una versión almacenada de la red diseñada, si esta no existe, se procede a entrenar

In [ ]:
try:

    transfered_net = nn.Sequential(steps)
    transfered_net.load_state_dict(torch.load('models/trasnfered_net.model'))
    transfered_net.eval()

except:

    bs = 100
    epochs = 100
    leaps = 1

    train_dl = DataLoader(train_ds, batch_size=bs,
                          shuffle=True, num_workers=4, pin_memory=True)
    valid_dl = DataLoader(valid_ds, batch_size=bs,
                          shuffle=True, num_workers=4, pin_memory=True)

    train_dl = ExtendedLoader(train_dl, transformacion_2)
    valid_dl = ExtendedLoader(valid_dl, transformacion_2)

    learning_data = fit(epochs,
                        transfered_net,
                        loss_func,
                        opt,
                        train_dl,
                        valid_dl,
                        metric=accuracy,
                        only_print=False,
                        print_leap=leaps)

se observan los resultados por medio de una curva de aprendizaje 

In [ ]:
try:
    
    learning_data = pd.read_csv('reports/learning_data.csv')
    show_learning_curves(learning_data)

except:

    show_learning_curves(learning_data, leaps=10)

con lo que se observa una capacidad razonable de generalización con una exactitud en torno a ... Se procede a estudiar la capacidad real de generalizacón en el conjunto test inicialmente cargado.

In [ ]:
bs = 250
test_dl = DataLoader(cifar_test, batch_size=bs,
                     shuffle=True, num_workers=4, pin_memory=True)

def predictor(y_hat): return torch.argmax(
    torch.softmax(y_hat, dim=1), dim=1)


data = np.zeros([len(cifar_test),2])
i = 0

transfered_net.cuda()
with torch.no_grad():
    
    transfered_net.eval()
    for xb, yb in test_dl:
        
        xb = xb.to(torch.float16)
        xb = xb.cuda()

        yb = yb.cpu().numpy()

        y_hat = transfered_net(xb)
        y_hat = predictor(y_hat).cpu().numpy()
        
        data[bs*i:bs*(i+1),0] = yb
        data[bs*i:bs*(i+1),1] = y_hat
        
        i+=1 

preds = pd.DataFrame(data,columns=['y_true','y_pred'])

por ultimo se construye la tabla de contingencia asociada.

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(**preds))

lo cual confirma una buena capacidad de generalización. 

**Ejercicio**

1. Utilice el atributo `.class_to_idx` del conjunto de datos `cifar_test` para descubrir cuales son los objetos mejor y peor aprendidos por la red. 

### Arquitecturas Conocidas de Red

Conocidas algunas formas de transferencia de aprendizaje, se pasan a estudiar redes pre-entrenadas y sus arquitecturas asociadas. 

Dentro de las arquitecturas de red encontramos:

**VGG**

Esta red hereda su nombre del grupo que la construye (Visual Geometry Group). Fue introducida en el 2014 y fue entrenada sobre *ImageNet*. La arquitectura de esta red se basa en la observación de que un filtro convolucional de gran tamaño / campo receptivo, puede ser reemplazada con una concatenación de dos o más capas convolucionales con filtros más pequeños. Por ejemplo, una capa de filtros de 5x5 puede reemplazada por dos capas con filtros de 3x3. 

Para comprender la idea tras esta afirmación, recordemos que el volumen de salida en términos de largo ($W_1$) y alto ($H_1$) para un volumen inicial de $(W_0,H_0)$ al cual se le aplica una capa convolucional con un kernel de dimensión $(K_w, K_h)$ con paddin $P$ y stride $S$ viene dado por:

$$
W_1=\frac{W_0-k_w+2 P}{S}+1
$$
$$
H_1=\frac{H_0-k_h+2 P}{S}+1
$$

Así una imagen en escala de grises con dimensiones (256,256,1), a la cual se aplica un filtro de 5x5 con $P=0$ y $S=1$ genera un volumen output de dimensiones: 

In [ ]:
p = 0 
s = 1 
w = 256 
k = 5

output_5 = int((w-k + 2*p)/s + 1)

print('Dimension output:', output_5)

donde cada uno de los {{output_5}} elementos tiene un campo receptivo de 5x5 píxeles en el volumen inicial. 

El proceso anterior puede ser emulado utilizando primero un kernel de 3x3 

In [ ]:
p = 0 
s = 1 
w = 256 
k = 3 

output_31 = int((w-k + 2*p)/s + 1)

print('Dimension output kernel 1:', output_31)

que da como resultado un volumen output con {{output_31}} elementos, cada uno de estos elementos tiene un campo receptivo de 3x3 pixeles en el volumen inicial. si posteriormente aplicamos una nueva capa con un filtro de 3x3 se obtiene un segundo volumen output

In [ ]:
p = 0 
s = 1 
w = 256 
k = 3 

output_32 = int((w-k + 2*p)/s + 1)

print('Dimension output kernel 1:', output_32)

este segundo volumen tiene {{output_32}} elementos, igual que al aplicar el filtro de 5x5 en el volumen inicial. En este caso, cada elemento de esta última capa output tiene como campo receptivo 3x3 elementos de la capa output anterior. Sin embargo, al contar el campo receptivo de esta última capa en la capa input se observar que se tiene un campo receptivo de 5x5 píxeles. 

En resumen, aplicar dos capas de 3x3 (18 pesos en total) consecutivamente emula el campo receptivo de aplicar un filtro de 5x5 (25 pesos en total) pero disminuye la cantidad de pesos a entrenar. 

Las redes VGG constan de múltiples bloques de a 2 hasta 4 capas convolucionales concatenadas, seguidas de una capa de pooling por máximización. Las más populares son VGG16 y VGG19. 

**Ejemplo**

La estructura de la red VGG16 se puede apreciar al importarla desde Pytorch:

In [ ]:
import torchvision.models as models
from torchsummary import summary

vgg16 = models.vgg16(pretrained=True)

utilizamos `summary` del paquete `torchsummary`:

In [ ]:
summary(vgg16,(3,224,244), device='cpu')

[Más información de la red](https://arxiv.org/abs/1409.1556)

**Redes Inception**

Esta red ganó el desafío de *ImageNet* en el año 2014, existen múltiples versiones de este tipo de arquitectura. La idea que fundamenta estas redes se basa en la idea de capturar objetos teniendo en cuenta su dimensión en la imagen. En general, las redes convolucionales estándar pueden capturar objetos a cierta escala fija, debido a la poca flexibilidad de los campos receptivos. 

Para resolver el problema de escala antes mencionado, se propuso una arquitectura compuesta por bloques de tipo *inception*, estos consisten en capas paralelas (o torres), todas las capas paralelas están conectadas a un mismo input. La idea de estas capas paralelas es generar filtros con distintos campos receptivos, finalmente los resultados de cada capa es concatenado al resto generando un volumen output. 

En la primera versión de la red inception se modelaban los bloques de capas paralelas según:

1. Capa de convolución 1x1.
2. Capa de convolución 1x1 seguida de una capa de 3x3.
3. Capa de convolución de 1x1 seguida por una capa de 5x5.
4. Capa de pooling máximo 3x3 con stride 1. 

Las capas de este bloque agregan padding de manera tal que el input y ouput tengan la misma dimensión espacial, pero distinta cantidad de canales. La ventaja de usar capas de 1x1 como componente del bloque radica en que permiten una reduccción de profundidad en un siguiente bloque inception, permitiendo conservar recursos (en caso contrario, la profundidad solo aumenta). 

Las siguientes versiones de la red (v2 y v3) se basan en la *factorización* de capas convolucionales introducidas por las redes VGG, sin embargo, acá se factorizaron filtro de $n \times n$ ($n^2$) elementos por dos capas de $1 \times n$ y $n \times 1$ respectivamente ($2n$ parámetros).

**Ejercicio**

1. Importe una red inception con Pytorch y estudie su arquitectura. 
2. ¿Es posible combinar la arquitectura inception con la de ResNet?¿qué ventajas tendría esta técinca? busque si ya existe una arquitectura que utilice esta idea.

Más información sobre estas redes: [1](https://arxiv.org/pdf/1409.4842v1.pdf), [2](https://arxiv.org/abs/1512.00567).

**Redes Xception**

Los bloques inception utilizan una reducción de dimensionalidad dada por convoluciones 1x1, esto reconoce correlaciones entre canales pero no obtiene información espacial, Por otra parte, las convoluciones siguientes obtienen información espacial, de esta forma se capturan correlaciones tanto espaciales como de color. 

Las redes Xception buscan desacoplar las relaciones entre canal (color) de las espaciales. Para ello se utilizan convoluciones en profundidad (*dephtwise*) separables. 

Una convolución separable depthwise combina dos operaciones, una convolución en los canales (profundidad) junto a una convolución 1x1. En el primer caso, se aplica un filtro por canal generando una lamina de salida por canal. Posteriormente, se aplican tantos filtros de 1x1 como profundad se busca en el volumen de salida. 

Se dice que las convoluciones depthwise son un caso *extremo* de un bloque inception pues se tienen tantas capas paralelas como combinaciones depthwise + 1x1 convoluciones se escojan. Se observa ademas que la convolución 1x1 viene luego de aplicar una capa convolucional, en el caso de la red inception ese proceso es al revés.  
[Más información de la red](https://arxiv.org/abs/1610.02357).

## Autoencoders y GAN's

Dentro de los modelos basados en redes neuronales para aprendizaje no supervisado, se encuentran los modelos generativos. 

Un modelo generativo intenta aprender la distribución de clases, en vez de predecir la probabilidad $P(\mathbf{y} |X=x)$, intenta predecir $P(X | \mathbf{y} = y)$. Es decir, busca la distribución del input según clase. Dos de los modelos más utilizados en este contexto son los *autoencoders variacionales* y las redes *generarativas adversariales*.


### Autoencoders Variacionales

Un autoencoder es una red neuronal feedforward cuya finalidad es reprodudir su input, es decir, el valor objetivo (target) corresponde a la misma observación que el modelo procesa. De esta forma se intenta aprender una función identidad $h_w(x) = x$, para encontrar invarianzas, la arquitectura de la red autoencoder debe modificar la dimensión del input, esto se hace por medio de capas *bottleneck* que reducen la dimensión de la capa anterior de la red. En terminos generales se puede pensar en una red autoencoder como una composición de un **encoder**, este consiste en una aplicación que toma datos input y los lleva a una representación distinta por medio de una arquitectura de red. El segundo componente es el **decoder** que consiste en una segunda arquitectura de red que intenta reconstruir los datos input tomando como punto inicial el output de la arquitectura *encoder*. No hay motivos para que ambas estructuras sean identicas (una como relfejo de la otra) aunque por lo general se implementa de esa manera. 

Para entrenar un autoencoder se minimiza una función de perdida $L(x,x')$ conocida como *error de reconstrucción* y puede ser minizada de la manera usual por medio de SGD por ejemplo. 

Como se mencionó, un autoencoder corresponde a un algoritmo no supervisado de aprendizaje basado en redes neuronales por lo tanto, su riqueza no se encuentra en una posible discriminación de inputs, sino más bien en la **representación latente** que permiten del espacio de entrada. Esta representación (dada por el encoder asociado a la red) permite aprender caracterísitcas del conjunto de datos y además reducir la dimensión de este, generando una *compresión* de la información contenida.  
**Ejemplo**

Se implementa un autoencoder sobre la base MNIST. Para ello, se define su arquitectura.

In [ ]:
import torch
import torch.utils.data 

import torchvision

from torchvision import datasets, transforms

from torch import nn, optim
from torch.nn import functional as F
from torchvision.utils import save_image


class AutoEncoder(nn.Module):
    '''Autoencoder Ejmplo sobre MNIST.'''

    def __init__(self, **kwargs):

        super().__init__()

        # Encoder
        self.encoder_fc1 = nn.Linear(
            in_features=28*28, out_features=128
        )
        self.encoder_fc2 = nn.Linear(
            in_features=128, out_features=128
        )

        # Decoder
        self.decoder_fc1 = nn.Linear(
            in_features=128, out_features=128
        )
        self.decoder_output = nn.Linear(
            in_features=128, out_features=28*28
        )

    def forward(self, xb):
        xb = self.encoder_fc1(xb)
        xb = torch.relu(xb)
        
        x_encoded = self.encoder_fc2(xb)
        x_encoded = torch.relu(x_encoded)
        
        x_decoded = self.decoder_fc1(x_encoded)
        x_decoded = torch.relu(x_decoded)
        
        x_decoded = self.decoder_output(x_decoded)
        
        x_reconstruction = torch.relu(x_decoded)
        
        return x_reconstruction

La red actúa como una red feedforward sobre la imagen de 28x28 representada como un vector de 784 componentes, posteriormente reduce la dimensión a 128 componentes, mantiene la representación y la vuelve a expandir a un vector de 784 componentes. Se implementa el modelo, su optimizador y pérdida asociada.

In [ ]:
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
model = AutoEncoder().to(dev)

opt = optim.Adam(model.parameters(), lr=1e-3)
loss_func = nn.MSELoss()

Finalmente se implementa un esquema de entrenamiento

In [ ]:
# Carga de datos
bs = 128
epochs = 20

transform = transforms.Compose([torchvision.transforms.ToTensor()])

train_ds = torchvision.datasets.MNIST(
    root='data/', train=True, transform=transform, download=True
)

test_ds = torchvision.datasets.MNIST(
    root="data/", train=False, transform=transform, download=True
)

# Encapsulacion de obtencion de dato mnist
def get_data_mnist(train_ds=train_ds, test_ds=test_ds, bs=bs):

    train_dl = torch.utils.data.DataLoader(
        train_ds, batch_size=bs, shuffle=True, num_workers=4, pin_memory=True
    )

    test_dl = torch.utils.data.DataLoader(
        test_ds, batch_size=int(bs/4), shuffle=False, num_workers=4
    )

    return train_dl, test_dl

se procede a entrenar

In [ ]:
train_dl, _ = get_data_mnist()

# Entrenamiento
for epoch in range(epochs):
    loss = 0

    for xb, _ in train_dl:

        # reshape a [N, 784]
        xb = xb.view(-1, 784).to(dev)

        opt.zero_grad()

        # reconstruye
        outputs = model(xb)

        train_loss = loss_func(outputs, xb)

        # backprop
        train_loss.backward()
        opt.step()

        # acumula la perdida
        loss += train_loss.item()

    loss = loss / len(train_loader)

    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

Se prueban algunas imagenes del conjunto test

In [ ]:
print('Imagen original:')

idx =  0 

ex = test_ds[idx][0] 
img = transforms.ToPILImage()(ex).resize([150,150]) 
img 

se reconstruye la imagen anterior

In [ ]:
print('Reconstruccion:')

ex = ex.view(-1, 784).to('cpu')
model.to('cpu')

with torch.no_grad():
    reconstruccion = model(ex)
    reconstruccion = reconstruccion.view([28, 28])

    reconstruccion = transforms.ToPILImage()(reconstruccion).resize([150, 150])

reconstruccion

Una variante del modelo autoencoder es su versión variacional. Esta realiza la representación latente de los datos input aproximando su distribución de probabilidad. Esto se puede formalizar de la siguiente manera:

* Se denota la sección encoder de la red como $q_{w} (z | x)$, donde $w$ son los parámetros de la red, $x$ es in input y $z$ una representación latente.

* Se denota el decoder como $p_{\theta} (x|z)$ donde $\theta$ son sus parámetros asociados. Acá, se obtiene una muestra de $z$ de manera aleatoria para luego ser decodificada, las salidas del proceso de muestreo y decodificación genera una distribución sobre los posibles valores de $x$. 

* Los autoencoders variacionales utilizan como función de pérdida la siguiente expresión:
$$
L(\theta, w ; x)=-D_{K L}\left(q_{w}(z \mid x) \| p_{\theta}(z)\right)+E_{q_{w}(z \mid x)}\left[\log \left(p_{\theta}(x \mid z)\right)\right]
$$

    Acá se busca minimizar la divergencia de Kullback-Leibler entre la distribución de probabilidad $q_w(z|x)$ y la probabilidad esperada $p_\theta (x)$. Esto busca modelar la pérdida de información al cambiar de la representación inicial a la parametrizada por $q$. El segundo término es la pérdida de reconstrucción entre el input original y la reconstrucción.
    
A modo de ejemplo, se puede implementar un autoencoder variacional donde la representación latente sea de la familia gaussiana, de esta forma, las capas del encoder entregarán como resultado un vector de media y uno de covarianza. Posteriormente, se procede a obtener muestras $z$ de la distribución normal parametrizada por tal media y covarianza para efectuar el proceso de decodificación. Para efectuar backpropagation, las muestras $z$ son obtenidas por medio sampleando un valor pivote $\epsilon \sim \mathcal{N}(0,1)$, de modo que $z = \mu + \sigma \cdot \epsilon$, donde $\mu$ y $\sigma$ son los parámetros obtenidos por la capa encoder. Así, se puede optimizar sobre los pesos de esta transformación afin, esto se conoce como **reparametrization trick**.

**Ejemplo**

Se construye un autoencoder variacional sobre la base MNIST, para ello, se hace la distinción entre los procesos de codificación (encode) y decodificación (decode).

In [ ]:
class VAE(nn.Module):
    def __init__(self, dev): 
        super(VAE, self).__init__()
        
        self.device = dev 
        
        # Encoder
        self.fc1 = nn.Linear(28*28, 400)
        self.relu = nn.ReLU()

        # Se aprende una representacion de media y varianza para 20 comps.
        self.fc21 = nn.Linear(400, 20)  # mu
        self.fc22 = nn.Linear(400, 20)  # log-varianza

        # Decoder
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)
        self.sigmoid = nn.Sigmoid()  # prob.

    def encode(self, x):
        h1 = self.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def rep_trick(self, mu, logvar):
        if self.training:
            # Se recupera la desviacion desde la log-varianza
            std = logvar.mul(0.5).exp_()

            # Se obtiene la muestra normal (0,1)
            eps = torch.randn(size=std.size(), device = dev)

            # Se retorna la muesta z
            return eps.mul(std).add_(mu)

        else:
           # Si no es entrena, la muestra a entregar sera z = mu
            return mu

    def decode(self, z):
        h3 = self.relu(self.fc3(z))
        return self.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.rep_trick(mu, logvar)
        return self.decode(z), mu, logvar

Se inicializa el modelo 

In [ ]:
model = VAE(dev)  
model.to(dev) 

Se define la función de pérdida

In [ ]:
def loss_function(recon_x, x, mu, logvar , bs = bs):
    
    # Error de reconstruccion
    rec_error = F.binary_cross_entropy(recon_x, x.view(-1, 784))
    
    # KL-D para el caso gaussiano
    # - D_{KL} = 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    
    kl_div = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    # normalizacion para coindidir con los ejemplos usados en reconstruccion
    kl_div /= bs * 784

    return kl_div + rec_error

se inicializa el optimizador

In [ ]:
opt = optim.Adam(model.parameters(), lr=1e-3)

se define un proceso de entrenamiento

In [ ]:
train_dl, _ = get_data_mnist()

# Entrenamiento
for epoch in range(epochs):
    loss = 0

    for xb, _ in train_dl:

        # reshape a [N, 784]
        xb = xb.view(-1, 784).to(dev)

        opt.zero_grad()

        # reconstruye
        outputs = model(xb)

        train_loss = loss_function(outputs[0], xb, outputs[1], outputs[2])

        # backprop
        train_loss.backward()
        opt.step()

        # acumula la perdida
        loss += train_loss.item()

    loss = loss / len(train_loader)

    print("epoch : {}/{}, loss = {:.6f}".format(epoch + 1, epochs, loss))

In [ ]:
print('Imagen original:')

idx =  0 

ex = test_ds[idx][0] 
img = transforms.ToPILImage()(ex).resize([150,150]) 
img 

In [ ]:
print('Reconstruccion:')

ex = ex.view(-1, 784).to(dev)
model.to(dev)

with torch.no_grad():
    reconstruccion = model(ex)
    reconstruccion = reconstruccion[0].view([28, 28]).to('cpu')
    reconstruccion = transforms.ToPILImage()(reconstruccion).resize([150, 150])

reconstruccion

[más información sobre VAE's](https://arxiv.org/abs/1312.6114)

## Redes Generativas Adversariales 

Una arquitectura de red generativa adversarial (GAN) consta de dos componentes: 

* Generador: Corresponde a un bloque de la red dedicado a generar ejemplos. Toma como input una distribución de probabilidad (por ejemplo ruido blanco) e intenta generar una salida ajustada al dataset que se trabaja. Esta sección de la red es similar al bloque codificador (encoder) de un VAE. 

* Discriminador: Consiste de dos inputs, un elemento del conjunto de datos y una muestra producida por el el bloque generador. Esta sección de la red intenta determinar si la observación generada corresponde a una observación real. 

Ambos bloques son entrenados en conjunto formando una red generativa adversarial. La idea de fondo es que el bloque discriminador busca mejorar en su capacidad de distinguir elementos generados de reales, mientras que el bloque generador intenta generar muestras cada vez más realistas "engañando" al bloque discriminador. El sistema en general busca producir un bloque generador tan eficiente, que el bloque discriminador no puede distinguir entre las muestras que produce. Aunque el bloque discriminador permite clasificar muestras, una arquitectura GAN corresponde a un modelo de aprendizaje no supervisado. 

Para entrenar una arquitectura GAN, se comienza por la sección generador, denotada por $G(z, \theta_g)$, donde $\theta_g$ corresponde a los pesos de la red generadora, $z$ es un vector latente asociado  y sirve como input para la red. El generador toma muestras $x$ según una distribución de probabilidad $p_g(x)$. Se puede pensar en esta probabilidad con la cual se distribuyen los datos de entrenamiento según el generador. 

El bloque discriminador se denota por $D(x,\theta_d)$ toma inputs reales $x \sim p_{data} (x)$ o generadas $p_g(x)$. El discriminador es un clasificador binario. 

Durante el entrenamiento se requieren funciones de pérdida tanto para el generador $J^{G}$ como para el discriminador $J^{D}$. El proceso de entrenamiento es distinto al de las redes convolucionales pues básicamente se poseen dos redes. Estas se relacionan según un juego secuencial minimax de suma cero entre dos jugadores. Acá secuencial se refiere a que los jugadores toman sus turnos de manera ordenada, es decir, primero el discriminador intenta minimizar $J^{D}$, para que luego el generador optimice $J^{G}$. Que el juego sea de suma cero, significa que las ganancias (o pérdidas) de un jugador se corresponden con las pérdidas (ganancias) del otro. Luego se espera que $J^{G} = - J^{D$. Por último, un juego minimax quiere decir que la estrategia de un jugador (generador) es minimzar el puntaje máximo de su adversario (discriminador). Cuando se entrena el discrimindor, este mejora en su capacidad de reconocer ejemplos falsos, mientras que al entrenar el generador se intenta producir una mejora en os pesos, de manera tal que se sobrepase la mejora del discriminador. Por lo anterior, ambos bloques se encuentran en constante competencia.

Cuando el esquema de aprendizaje las pérdidas $J^{D}$ y $J^{G}$ alcanzan minimos locales, se dice que la solución del problema minimaz alcanza un equilibrio de Nash. Acá ninguno de los jugadores cambia sus jugadas, sin importar lo que haga su oponente. En este escenario, el discriminador tendrá como output el valor $0.5$ sin importar el input que recibe. 




## Redes Neuronales Recurrentes

Las redes neuronales descritas hasta el momento tienen un input de tamaño fijo para le cual entregan un output de tamaño fijo. Una **red neuronal recurrente** RNN permite procesar datos de entrada secuenciales de tamaño variable, un ejemplo emblemático es el procesamiento de lenguaje natural NLP. La secuencialidad de los datos en este ejemplo juega un rol fundamental pues implica que los elementos que la componen están relacionados entre si, y por tanto, su orden debe ser capturado. 

Las redes neuronales recurrentes aplican una función de manera recursiva sobre una secuencia input. Esto se puede definir según la relación $s_t = f(s_{t-1}, x_t)$ Acá $f$ es una función diferenciable, $s_t$ es un vector de valores denominado *estado interno de la red en el tiempo $t$* y $x_t$ es el input de la red en el tiempo $t$. A diferencia de las redes ya vistas, donde el estado de la red depende solo del input y los pesos de la red, con $s_t$ se modela una relación entre el input actual y el estado anterior $s_{t-1}$. La relación de recurrencia define como el estado de la red evoluciona paso a paso sobre una secuencia utilizando ciclos de retroalimentación sobre estados anteriores. Una red neuronal recurrente tiene tres tipos de parámetros: 

1. $U$ transforma el input $x_t$ al estado $s_t$.
2. $W$ transforma el estado previo $s_{t-1}$ en el estado actual $s_t$.
3. $V$ transforma el estado actual a un output $y_t$.

En este caso, $U,V,W$ aplican transformaciones lineales sobre sus respectivos inputs. Lo anterior se puede visualizar utilizando como transformación una suma ponderada, de esta manera el estado interno de la red asociada a esta construcción pasa a ser:
\begin{align}
s_{t}&=f\left(s_{t-1} * W+x_{t} * U\right) \\
y_{t}&=s_{t} * V
\end{align}

Acá $f$ es una función de activación no lineal, como las ya utilizadas hasta el momento. 

A modo de ejemplo, se puede considerar un modelo de texto, aquí, el input $x$ será una secuencua de palabras codificada en vectores de la forma $(x_1, x_2, \ldots, x_t, \ldots)$. El estado $s$ será una secuencia de vectores de estado $(s_1,s_2, \ldots, s_t, \ldots)$. Por último, el output de la red $y$ será una secuencia de vectores de probabilidad $(y_1, \ldots, y_t, \ldots )$ para la siguiente palabra de la secuencia. 

Si se observa la RNN básica descrita anteriormente, se puede apreciar una estecha similitud con una capa de un perceptron multi-capa. Utilizando el mismo principio de composición de capas, se puede crear una RNN como resultado de composiciones entre múltiples redes recurrentes con una arquitectura similar a la discutida. En este contexto, se trabaja con estados del tipo $s_t^l$ que hacen referencia al nivel o capa $l$ de la composición de estructuras en el tiempo $t$. En tal nivel, se toma como input un arreglo de la forma $y_{t}^{l-1}$ correspondiente al output de la capa anterior utilizando el estado $s_{t-1}^{l}$. Debido a que las RNN's no tienen restricciones en su input ni output, aparecen múltiples escenarios de aplicación, se puede por ejemplo crear una red recurrente **uno a uno**, en este caso la red recurrente se reduce a una red tipo feedforward como el perceptron multi-capa o una red convolucional, en este caso no se procesan secuencias input. Se puede crea una red **uno a muchos**, acá la red recibe input no secuencial, entregando secuencias como resultados (ej: generar tags de imagenes). El caso opuesto al anterior es **muchos a uno**, donde se procesan secuencias que tienen como resultados ouptuts singulares (ej: clasificación de texto). Por último aparecen las relaciones **muchos a muchos**, estas pueden ser *indirectas*, donde una secuencia es codificada a un vector de estado para luego decodificar tal estado en una nueva secuencia (ej: traducción de textos), la última interacción es de ltipo *directa*, donde las salidas son obtenidas a partir de cada input en cada instante (ej: reconocimiento de voz). 


**Ejemplo**

Se procede a modelar una RNN para una tarea sencilla, esto se hace utilizando unicamente NumPy. 

In [ ]:
import numpy as np

el problema a modelar consiste en contar la cantidad de elementos no nulos en una secuencia binaria. La red a modelar tendrá solo dos parámetros, un peso input $U$ y un peso recurrente $W$, el peso output $W$ será 1.Generamos la secuencia inicial y la respuesta correspondiente:

In [ ]:
n = 10
p = 0.7

x = np.random.binomial(1,p, n) 
y = np.sum(x) 

print('x generado:', x) 
print('Respuesta asociada:', y)

La relación de recurrencia se define según $s_{t}=s_{t-1}^{*} W+x_{t}^{*} U$, en este caso, la función de activación es la identidad. Se procede a implementar la relación de recurrencia

In [ ]:
def rec_step(s, x, U, W):
    return x * U + s * W

Si hacemos $U = 1$, entonces siempre que se recibe un input, se obtiene directamente su valor. Por otra parte, si $W = 1$ el valor obtenido dle input se irá acumulando sin caer. En este caso, se debería por tanto tener el resultado esperado:

In [ ]:
leap = 3
s = 0
for t in range(n): 
    s = rec_step(s,x[t],1,1)
    
    if t % leap == 0:
        print('Output Real:', np.sum(x[:t])) 
        print('Output Red: ', s , '\n')  

Se tiene así que los valores optimos para esta red son $U=1$ y $W=1$. 

### Backpropagation en Redes Recurrentes 

El algoritmo de Backpropagation *en el tiempo* es un algoritmo tipico de entrenamiento en RNN's. La diferencia principal entre la versión recurrente y la normal, es que en el primer caso, la red es *desenvuelta* ciero numero de pasos temporales. La idea de este proceso es generar una *sub-red* no recurrente, así, cada capa de esta *sub-red* representa un paso temporal con múltiples inputs, dados por el estado previo $s_{t-1}$ y el estado actual $x_t$. Los parámetros $U$ y $W$ se comparten en cada capa oculta de esta red. 

Por otra parte, el método *forward* aplica la RNN sobre una secuencia generando una pila de estados para cada paso. 

**Ejemplo**

Se procede a implementar el paso *forward* para una red recurrente simple:

In [ ]:
def forward_step(x, U, W):
    # logitud de mini-batch -> cantidad de secuencias
    number_of_samples = len(x)
    
    # longitud de cada muestra en el batch
    sequence_length = len(x[0])
    
    # Se inicializa el estado de activiacion para cada muestra
    s = np.zeros((number_of_samples, sequence_length + 1))
    
    # Se actualizan los estados
    for t in range(0, sequence_length):
        
        '''
        Se usa la red (rec_step) en cada paso:
        
        s_1 = rec_step(s_1, x_1, U ,W)
        '''
        
        s[:, t + 1] = rec_step(s[:, t], x[:, t], U, W)
    
    return s 

Se comprueba su funcionamiento, observe que $s_0$ es en este caso la segunda entrada del vector que almacena los estados. 

In [ ]:
W = 1
U = 1

x = x.reshape([1,-1])
print('Secuencia entregada:', *x)  

print('Estado de la red: ', forward_step(x,U,W))

Dado le paso forward, se requiere definir una función de pérdida, en este ejemplo podemos utilizar el erro cuadrático:

In [ ]:
loss_func = lambda x,y : (x-y)**2

El siguiente paso es entregar un método de propagación hacia atrás, este debe capturar la idea de *desenvolver* la red recurrente. Dado que los pesos $U$, $W$ son compartidos entre capas de la *sub-red* desenvuelta, se acumularán las derivadas del error en cada paso recurrente, al final del proceso se adaptan los pesos con el valor acumulado.

El primer paso es entonces obtener los gradientes de los outputs $s_t$ con respecto a la función de pérdida $J$, para luego propagar hacia atrás a través de los pasos realizados en el paso *forward*. La relación de recurrencia para propagar el gradiente por la red se puede escribir utilizando la regla de la cadena:

$$
\frac{\partial J}{\partial s_{t-1}}=\frac{\partial J}{\partial s_{t}} \frac{\partial s_{t}}{\partial s_{t-1}}=\frac{\partial J}{\partial s_{t}} W
$$

Los gradientes de los parámetros se acumulan según: 
$$
\frac{\partial J}{\partial U}=\sum_{t=0}^{n} \frac{\partial J}{\partial s_{t}} x_{t}
$$
y

$$
\frac{\partial J}{\partial W}=\sum_{t=0}^{n} \frac{\partial J}{\partial s_{t}} s_{t-1}
$$

Se procede a acumular los gradientes de $U$ y $W$:

In [ ]:
def backward_step(x, s, y, W):

    sequence_length = len(x[0])

    # Se calcula el output y -> ultimo estado de la red
    s_t = s[:, -1]

    '''
    Se calcula el gradiente del output, se usa el error 
    cuadratico como perdida.
    '''
    
    grad_S = 2*(s_t - y)

    # Se comienza a acumular **hacia atras**
    grad_U, grad_W = 0, 0

    for k in range(sequence_length, 0, -1):

        grad_U += np.sum(grad_S * x[:, k - 1])
        grad_W += np.sum(grad_S * s[:, k - 1])

        # Se calcula el gradiente del output en la capa anterior
        grad_S = grad_S * W

    return grad_U, grad_W

Se pueden utilizar los gradientes calculados para entrenar nuestra red: 

In [ ]:
def train(x, y, epochs, learning_rate=0.0005 , weights = (-2, 0)):

    # Acumuladores de pesos, y perdidas
    losses = list()
    weights_u = list()
    weights_w = list()

    # Descenso de gradiente
    for i in range(epochs):

        s = forward_step(x, weights[0], weights[1])

        loss = loss_func(y, s[-1, -1])

        # Se almacenan los valores
        losses.append(loss)
        weights_u.append(weights[0])
        weights_w.append(weights[1])

        # Se calcula el paso backward
        gradients = backward_step(x, s, y, weights[1])

        # p -> p - (gradient *learning_rate).
        weights = tuple((p - gp * learning_rate)
                        for p, gp in zip(weights, gradients))
        
    print(weights)
    return np.array(losses), np.array(weights_u), np.array(weights_w)

Se implementa una función para visualizar los pesos y la función de pérdida.

In [ ]:
import matplotlib.pyplot as plt

def plot_training(losses, weights_u, weights_w):
    
    # Elimina nans
    losses = losses[~np.isnan(losses)][:-1]
    weights_u = weights_u[~np.isnan(weights_u)][:-1]
    weights_w = weights_w[~np.isnan(weights_w)][:-1]
    
    # Grafica pesos U and W
    fig, ax1 = plt.subplots(figsize=(9, 7))
    ax1.set_ylim(-3, 2)
    ax1.set_xlabel('epochs')
    ax1.plot(weights_w, label='W', color='red', linestyle='--')
    ax1.plot(weights_u, label='U', color='blue', linestyle=':')
    ax1.legend(loc='upper left')
    
    # Otra axis con las mismas dimensiones
    ax2 = ax1.twinx()
    
    ax2.set_ylim(-3, 200)
    ax2.plot(losses, label='Loss', color='green')
    ax2.tick_params(axis='y', labelcolor='green')
    ax2.legend(loc='upper right')
    fig.tight_layout()
    plt.show()

Se procede a entrenar la red construida

In [ ]:
np.random.seed(6202) 

n = 10
p = 0.7

x = np.random.binomial(1,p, [1,n])  
y = np.sum(x) 

print('x generado:', x) 
print('Respuesta asociada:', y)

In [ ]:
losses, weights_u, weights_w = train(x, y, epochs=150, weights= (-2,1)) 
plot_training(losses, weights_u, weights_w)

Esto muestra cierto cambio en los pesos, al correr el ejemplo con una secuencia más grande se obtienen errores de *gradientes que explotan*

In [ ]:
n = 15   
p = 0.7

x = x = np.array([[0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
0]])

y = 12 

losses, weights_u, weights_w = train(x, y, epochs=150) 

esto se visualiza en el siguiente gráfico

In [ ]:
plot_training(losses, weights_u, weights_w)

Lo que ocurre es que los pesos se mueven a un mínimo local gradualmente, hasta que se disparan en la epoca 23, lo cual se debe a la estructura recurrente de la red y a la inestabilidad en los gradientes de la red en función de la función de costo. Esto se conoce como **explosión de gradientes**. 

Existe tambien el caso opuesto de **desaparición de gradientes**, en este caso, los gradientes decaen de manera rápida. En este caso, la red va perdiendo la habilidad de guardar estados pasados y seguirá generando outputs validos (a diferencia de los gradientes que explotan). 

Aunque estos fenómenos estan presentes en redes neuronales clásicas, en el caso de las RNN's se exacerban principalmente pues el proceso de *desenvolver la red* puede ser muy profundo (en función del largo de la secuencia) y por que además los pesos son compartidos en todos los pasos de la red en cada paso, esto en conjunción con la relación de recurrencia, hace que se forme una sucesión geometrica en el paso de propagación hacia atrás:

$$
\frac{\partial s_{t}}{\partial s_{t-m}}=\frac{\frac{\partial s_{t}}{\partial s_{t-1}} * \ldots * \partial s_{t-m+1}}{\partial s_{t-m}}=W^{m}
$$

En el ejemplo analizado, el gradiente crece de manera exponencial si $|W > 1$. Por otra parte, si $|W|<1$ se cae en el caso de gradientes que se anulan. En el caso general, cuando $W$ es una matriz, se tendrán gradientes que explotan o desaparecen en función del valor propio más grande de $W$ (radio espectral). 

### Redes LSTM

Para tratar los problemas de gradientes que explotan y se anulan, se crearon las redes recurrentes con memoria de corto plazo grande (long short-term memory). Las redes LSTM introducen la idea de *celda de estado*, donde la información puede solamente ser escrita o eliminada, de tal manera que el estado se mantiene constante si no hay estimulos exteriores. La celda de estado puede ser modificada por distintas *compuertas* o *gates* los cuales son una manera de hacer pasar información. Estas compuertas están compuestas por funciones logisticas sigmideas compuestas con multiplicaciones elemento por elemento. Como la función logistica solo tiene outputs entre 0 y 1, la multiplicación puede solo reducir el valor que circula por la compuerta. Una red LSTM tipica esta compuesta por 3 compuertas: compuerta de olvido (*forget gate*), compuerta input y compuerta output. La celda de estado, input y outputs son todos vectores, por lo que una red LSTM puede formarse por bloques de compuertas o componerse con otras arquitecturas.